In [2]:
import uproot
import sys
import glob
import os
import struct
import time
import csv
import pickle
import math
import gc

import numpy                  as np
import pandas                 as pd
import matplotlib             as mpl
import matplotlib.pyplot      as plt
import tensorflow             as tf
from   tensorflow             import keras

import scipy
from   scipy             import interpolate
from   scipy.interpolate import CubicSpline, splev, splrep, PPoly
from   scipy.optimize    import curve_fit, minimize

# from   scipy.stats            import kde
# from   mpl_toolkits           import mplot3d
# from   mpl_toolkits.mplot3d   import Axes3D

# sys.path.insert(0, "/home/chan/2_ROOT/0_FROST/0_FROSTpython")
# from   rbDST2dict import rbDST2dict_chan as frz
# from   rbDST2dict.rbDST2dict_chan import rbDST2dict_chan as frz
# from   rbDST2dict             import rbDST2dict_chan as frz      #rbDST2dict=dir, rbDST2dict.py=module, rbDST2dict()=class
                                                                 #you need to specify the folder -> "from rbDST2dict"

#To increase the cell width + load Images
from IPython.core.display import display, HTML, Image
# display(HTML("<style>.container { width:100% !important; }</style
#Pandas options
pd.options.display.max_columns = None
# pd.options.display.max_rows    = 100

#Plotting Config
mpl.style.use("default")
mpl.rcParams["axes.facecolor"] = "#EAEAF2" 
mpl.rcParams['figure.dpi']     = 100 
mpl.rcParams['savefig.dpi']    = 100 

# Read data

In [ ]:
#========== READ RAW - LOW ============================
#Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #========== READ RAW - HIGH ============================
# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# #Read RAW (WeiMome + HeBath + Fiducial + SOME COMPUTES)
# df_GMSH = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_raw.pkl")

# Event selections systematics

In [ ]:
#Low Momentum

#Drop <350 MeV
df_GMSH.drop(df_GMSH[df_GMSH["el_p_abs"] < 0.35].index, inplace=True)

#Drop <400 MeV
# df_GMSH.drop(df_GMSH[df_GMSH["el_p_abs"] < 0.4].index, inplace=True)

In [ ]:
#Proton Select

#Assume 3\sigma
three_Sig    = 0.06
sigma        = three_Sig / 3
two_five_Sig = sigma * (2.5)

#Current step
current_step = "proton_select"

#Systematic - New selection range - 2.5\sigma
df_GMSH.drop(df_GMSH[(df_GMSH["beta_diff"] < -two_five_Sig) | (df_GMSH["beta_diff"] > two_five_Sig)].index, \
             inplace=True)

In [ ]:
#Photon Select

#Drop events with bad photon and proton timings (MUST)
df_GMSH.drop(df_GMSH[df_GMSH["timediff"] > 1].index, inplace=True)
df_GMSH.drop(df_GMSH[df_GMSH["timediff"] < -1].index, inplace=True)

In [ ]:
# #TOF

#Run 1
df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==38) | (df_GMSH["tof_pad"]==44)))       | \
                     ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==44)))       | \
                     ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==32) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==36) | (df_GMSH["tof_pad"]==40) | (df_GMSH["tof_pad"]==42)))  | \
                     ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23))) | \
                     ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

# #Run 2
# df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==38) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==32) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==36) | (df_GMSH["tof_pad"]==40) | (df_GMSH["tof_pad"]==42)))  | \
#                      ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23))) | \
#                      ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

# #Run 3
# df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==38) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==32) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==36) | (df_GMSH["tof_pad"]==40) | (df_GMSH["tof_pad"]==42)))  | \
#                      ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23))) | \
#                      ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

# #Run 4
# df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==38) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==32) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==36) | (df_GMSH["tof_pad"]==40) | (df_GMSH["tof_pad"]==42)))  | \
#                      ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23))) | \
#                      ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

# #Run 5
# #Drop paddles (MUST)
# df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==38) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==44)))       | \
#                      ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==32) | (df_GMSH["tof_pad"]==35) | (df_GMSH["tof_pad"]==36) | (df_GMSH["tof_pad"]==40) | (df_GMSH["tof_pad"]==42)))  | \
#                      ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23))) | \
#                      ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

# #Run 6
# #Drop paddles (MUST)
# df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==1) & ((df_GMSH["tof_pad"]==17) | (df_GMSH["tof_pad"]==24) | (df_GMSH["tof_pad"]==26) | (df_GMSH["tof_pad"]==36)))       | \
#                      ((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==45)))       | \
#                      ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35)))       | \
#                      ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==42) | (df_GMSH["tof_pad"]==49)))  | \
#                      ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==52) | (df_GMSH["tof_pad"]==53)))       | \
#                      ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

# #Run 7
# #Drop paddles (MUST)
# df_GMSH.drop(df_GMSH[((df_GMSH["sector"]==1) & ((df_GMSH["tof_pad"]==17) | (df_GMSH["tof_pad"]==24) | (df_GMSH["tof_pad"]==26) | (df_GMSH["tof_pad"]==36)))       | \
#                      ((df_GMSH["sector"]==2) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==45)))       | \
#                      ((df_GMSH["sector"]==3) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==35)))       | \
#                      ((df_GMSH["sector"]==4) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==42) | (df_GMSH["tof_pad"]==49)))  | \
#                      ((df_GMSH["sector"]==5) & ((df_GMSH["tof_pad"]==23) | (df_GMSH["tof_pad"]==52) | (df_GMSH["tof_pad"]==53)))       | \
#                      ((df_GMSH["sector"]==6) & ((df_GMSH["tof_pad"]==44) | (df_GMSH["tof_pad"]==46)))].index, inplace=True)

In [ ]:
#ML Quick

def T_fin(z_vtx):
    tfin = 0
    #Butanol
    #    -All events left of 3cm vtx position
    if (z_vtx<=3.5):
        tfin = 0
        
#     #Ice
#     elif(T_pred == 1 and ice_pred == 0):
#         tfin = 3

    #Carbon
    #    -All events right of 3cm vtx position
    elif (z_vtx>3.5):
        tfin = 1
        
#     #Polythene
#     elif(T_pred == 2):
#         tfin = 2

    return tfin
    
#Vectorize
vect_T_fin             = np.vectorize(T_fin, otypes=[np.int])
T_fin                  = vect_T_fin(df_GMSH["z"].values)

#Append to df
df_GMSH["T_fin"] = pd.Series(T_fin, index=df_GMSH.index)

In [ ]:
#ZVTX

df_GMSH.drop(df_GMSH[(df_GMSH["z"] < -3.) | \
                     ((df_GMSH["z"] > 3.) & (df_GMSH["z"] < 5))   | \
                     (df_GMSH["z"] > 8)].index, inplace=True)

# Save init_plus

In [ ]:
#========== Save init_plus - LOW ============================
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#========== Save init_plus - High ============================
#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Save Initial PlUS (ML + zvtx + SF)
# df_GMSH.to_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# Read Bins + Combine df 

In [ ]:
#=============== Bins Read - LOW ======================================
#Read saved Bins
df_epho_dict = pd.read_csv("./results_Binning/epho_bins", index_col=0)
#Into epho_bins list
epho_bins = df_epho_dict[df_epho_dict.columns[0]].tolist()

print(epho_bins)

#Read cos_bins dict
df       = pd.read_csv("./results_Binning/cos_bins", index_col=0, delimiter="\t", header=None)
cos_bins = df.transpose().to_dict(orient="list")

#Print
for i in sorted(cos_bins.keys()):
    for j in range(len(cos_bins[i])):
        cos_bins[i] = [round(a, 4) for a in cos_bins[i]]
    print(i, cos_bins[i])
    
# #=============== Bins Read - HIGH ======================================
# #Read saved Bins
# df_epho_dict = pd.read_csv("./results_Binning/high_energy/epho_bins", index_col=0)
# #Into epho_bins list
# epho_bins = df_epho_dict[df_epho_dict.columns[0]].tolist()

# print(epho_bins)

# #Read cos_bins dict
# df       = pd.read_csv("./results_Binning/high_energy/cos_bins", index_col=0, delimiter="\t", header=None)
# cos_bins = df.transpose().to_dict(orient="list")

# #Print
# for i in sorted(cos_bins.keys()):
#     for j in range(len(cos_bins[i])):
#         cos_bins[i] = [round(a, 4) for a in cos_bins[i]]
#     print(i, cos_bins[i])

In [ ]:
#=============== Combined DF Read - LOW ======================================
#Read Initial PlUS (ML + zvtx + SF)
df_GMSH_1= pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_1/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_2 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_2/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_3 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_3/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_4 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_5 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_6 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_7 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/low_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Combine
df_GMSH = pd.concat([df_GMSH_1, df_GMSH_2, df_GMSH_3, df_GMSH_4, df_GMSH_5, df_GMSH_6, df_GMSH_7], \
                     ignore_index=True)

#Delete unused
del df_GMSH_1
del df_GMSH_2
del df_GMSH_3
del df_GMSH_4
del df_GMSH_5
del df_GMSH_6
del df_GMSH_7

# #=============== Combined DF Read - HIGH ======================================
# #Read Initial PlUS (ML* + zvtx + SF)
# df_GMSH_4 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Read Initial PlUS (ML* + zvtx + SF)
# df_GMSH_5 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Read Initial PlUS (ML* + zvtx + SF)
# df_GMSH_6 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Read Initial PlUS (ML* + zvtx + SF)
# df_GMSH_7 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

# #Combine
# df_GMSH = pd.DataFrame()
# df_GMSH = pd.concat([df_GMSH_4, df_GMSH_5, df_GMSH_6, df_GMSH_7], \
#                      ignore_index=True)

# #Delete unused
# del df_GMSH_4
# del df_GMSH_5
# del df_GMSH_6
# del df_GMSH_7

In [ ]:
#Drop - for SF and DF step

"""
-DONT DROP - doing HAE or mmsq step
-DROP      - if doing SF and DF step
"""
#Drop unneeded columns to speed up computation
column_drop_list = []
columns          = df_GMSH.columns
needed_columns   = ["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]
drop_list        = []

for i in columns:
    if (i not in needed_columns):
        drop_list.append(i)
        
#Drop-
df_GMSH.drop(columns=drop_list, axis=1, inplace=True)

# HAE Compute - LOW

#### SF

###### Partition - Butanol and Carbon in each bins (MUST)

In [ ]:
#Partition df in epho & cos bins
def part_df(df=pd.DataFrame(), epho_bins=[], cos_bins={}):
    #================================================================================================================
    #Dictionaries to data frames for each energy + angle bins
    buta_dict = {}
    carb_dict = {}
    
    #================================================================================================================
    #For each targets
    df_buta   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 0) & (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    df_carb   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 1) & (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]

    #================================================================================================================
    #For each Eg_bon
    for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
        df_buta_Eg = df_buta[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_buta["epho"] > epho_bins[i]) & (df_buta["epho"] < epho_bins[i+1])]
        
        df_carb_Eg = df_carb[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_carb["epho"] > epho_bins[i]) & (df_carb["epho"] < epho_bins[i+1])]
        
        #List of dataframes in same epho bins
        buta_epho_list = []
        carb_epho_list = []
        
        #========================================================================================================
        #For each cos_bin
        for k in range(len(cos_bins[j])-1):
            df_buta_Eg_cos = df_buta_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][k]) & (df_buta_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            
            df_carb_Eg_cos = df_carb_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_carb_Eg["cthe_cm"] > cos_bins[j][k]) & (df_carb_Eg["cthe_cm"] < cos_bins[j][k+1])]
        
            #REMOVE OUTLIER
            #    -For epho_bin=1, cos_bin==3, error: multiple copies of same event
            if (i==1 and k==3):
                df_carb_Eg_cos.drop(df_carb_Eg_cos[(df_carb_Eg_cos["el_pc_mmsq_pi0"]>-0.2837) & \
                                                   (df_carb_Eg_cos["el_pc_mmsq_pi0"] < -0.28365)].index, \
                                    inplace=True)
        
            #Append to its epho list
            buta_epho_list.append(df_buta_Eg_cos)
            carb_epho_list.append(df_carb_Eg_cos)
            
        #========================================================================================================            
        #Append to buta_dict & carb_dict for each energy bins
        buta_dict["Eg_{:07.3f}".format(epho_bins[i])] = buta_epho_list
        carb_dict["Eg_{:07.3f}".format(epho_bins[i])] = carb_epho_list            
        
    return buta_dict, carb_dict, epho_bins, cos_bins    

#Apply part_df
buta_dict, carb_dict, epho_bins, cos_bins = part_df(df_GMSH, epho_bins=epho_bins, cos_bins=cos_bins)

###### Initial SF function (MUST)

In [ ]:
#Scale Factor phase 1 (MUST)
#    -Returns # of butanol and # of carbon in each cos_bins and epho_bins for mmsq regions of interest 

#=================================================================================================================
#Selecton Range for SF calculation (i.e. [peak - \sig, peak - 2\sig])
#    -#THIS "sf_select_range" WILL BE DETERMINED BY NATURE OF FREE PEAK OF BUTANOL FITTING PARAMETERS
sf_select_range = (-2., -1.5)

#SF function
def init_scale_fact_1(df = pd.DataFrame(), epho_bins=[], cos_bins={}):
    
    #================================================================================================================
    #Dictionaries to hold scale factor for each energy + angle bins
    sf_1_dict          = {}
    
    #================================================================================================================
    #For each targets
    df_buta   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 0) & \
                                                                   (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    df_carb   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 1) & \
                                                                   (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    
    #================================================================================================================
    #For each Eg_bon
    for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
        df_buta_Eg = df_buta[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_buta["epho"] > epho_bins[i]) & (df_buta["epho"] < epho_bins[i+1])]
        
        df_carb_Eg = df_carb[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_carb["epho"] > epho_bins[i]) & (df_carb["epho"] < epho_bins[i+1])]
        
        #list: For each epho bin, containing a list for each angle bin
        sf_1epho_cos_list = []
        
        #========================================================================================================
        #For each cos_bin
        for k in range(len(cos_bins[j])-1):
            df_buta_Eg_cos = df_buta_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][k]) & (df_buta_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            df_carb_Eg_cos = df_carb_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_carb_Eg["cthe_cm"] > cos_bins[j][k]) & (df_carb_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            #inner list: For each angle bin, containg a scalar for each mmsq bin
            sf_1epho_1cos_mmsq_list = []
            
            #========================================================================================================
            #Histogram for MMSQ
            buta_data = np.array(df_buta_Eg_cos["el_pc_mmsq_pi0"])
            carb_data = np.array(df_carb_Eg_cos["el_pc_mmsq_pi0"])
            buta_mmsq_hist, buta_bins = np.histogram(buta_data, bins="fd")
            carb_mmsq_hist, carb_bins = np.histogram(carb_data, bins=buta_bins)
            
            #Find Peak of Butanol
            peak = buta_bins[np.argmax(buta_mmsq_hist)]
            
            #Find stddev of Butanol
            mids   = 0.5*(buta_bins[:-1] + buta_bins[1:])
            mean   = np.average(mids, weights=buta_mmsq_hist)
            var    = np.average((mids - mean)**2, weights=buta_mmsq_hist)
            stddev = np.sqrt(var)
            
            #=====================================================================================================
            #Find ranges of mmsq: [\sigma, 2*\sigma] with Butanol
            range_list = [peak + sf_select_range[0]*stddev, peak + sf_select_range[1]*stddev]
#             range_list_low = [peak + sf_select_range_low[0]*stddev, peak + sf_select_range_low[1]*stddev]
#             range_list_up  = [peak + sf_select_range_up[0]*stddev, peak + sf_select_range_up[1]*stddev]
            
            #Make New Histogram within defined ranges above: Events within [\sigma, 2*\sigma]
            n_b_i, buta_select_bins = np.histogram(buta_data[(buta_data>=range_list[0]) & (buta_data<=range_list[1])],\
                                                                        bins="fd")
            n_c_i, carb_select_bins = np.histogram(carb_data[(carb_data>=range_list[0]) & (carb_data<=range_list[1])], \
                                                                        bins=buta_select_bins)
#             n_b_i, buta_select_bins = np.histogram(buta_data[((buta_data>=range_list_low[0]) & (buta_data<=range_list_low[1])) | \
#                                                              ((buta_data>=range_list_up[0])  & (buta_data<=range_list_up[1]))], \
#                                                    bins="fd")
#             n_c_i, carb_select_bins = np.histogram(carb_data[((carb_data>=range_list_low[0]) & (carb_data<=range_list_low[1])) | \
#                                                              ((carb_data>=range_list_up[0])  & (carb_data<=range_list_up[1]))], \
#                                                    bins=buta_select_bins)
            
            #========================================================================================================    
            #Append n_b_i and n_c_i lists to sf_1epho_1cos_mmsq_list
            sf_1epho_1cos_mmsq_list.extend([n_b_i.tolist(), n_c_i.tolist()])
            
            #========================================================================================================    
            #Append sf_1epho_1cos_mmsq_list and to sf_1epho_cos_list
            sf_1epho_cos_list.append(sf_1epho_1cos_mmsq_list)

        #========================================================================================================            
        #Append to sf_eg_list to dict
        sf_1_dict["Eg_{:07.3f}".format(epho_bins[i])] = sf_1epho_cos_list
        
    return sf_1_dict, epho_bins, cos_bins

In [ ]:
#Defined Chi-Squared function (MUST)
def chi_sq_fn(x, n_b_i, n_c_i):
    #As defined in Nick's analysis paper
    chi_sq = np.sum(np.power(n_b_i - x*n_c_i, 2) / (n_b_i + (x**2)*n_c_i))
    return chi_sq

In [ ]:
#Scae factor phase 2 (MUST)
#    -Chi-Square miminization to get scale factor
def init_scale_fact_2(sf_1_dict={}, epho_bins=[], cos_bins={}):

    #==========================================================================================================
    #Dictionary to hold final sf values for each epho/cos bins
    sf_2_dict = {}
    
    #Keys of sf_1_dict corresponding to epho bins
    keys = sorted(sf_1_dict.keys())

    #==========================================================================================================
    #For each epho bins
#     for i in range(len(keys)):
    for i, j in zip(range(len(keys)), sorted(cos_bins.keys())):
        
        #list: For each epho bin, containing a list for each angle bin
        sf_1epho_cos_list = []
        
        #======================================================================================================
        #For each cos bins
        for k in range(len(cos_bins[j])-1):
    
            #Minimize using scipy
            result = scipy.optimize.minimize(chi_sq_fn, x0=5, \
                                             args=(np.array(sf_1_dict[keys[i]][k][0]), np.array(sf_1_dict[keys[i]][k][1])), \
                                             method="CG")
            
            if (result.success == True):
                sf = result.x[0]
            elif (result.success == False):
                #Set sf=5 just for now. Come back and change later
                sf = np.nan
            
            #Append final sf for each cos bins to sf_1epho_cos_lisabs
            sf_1epho_cos_list.append(sf)
    
        #==========================================================================================================
        sf_2_dict["Eg_{:07.3f}".format(epho_bins[i])] = sf_1epho_cos_list

    
    return sf_2_dict, epho_bins, cos_bins

In [ ]:
#Apply scale_fact_1 (MUST)
init_sf_1_dict, epho_bins, cos_bins = init_scale_fact_1(df_GMSH, epho_bins, cos_bins)

In [ ]:
#Call scale_fact_2(MUST)
init_sf_2_dict, epho_bins, cos_bins = init_scale_fact_2(init_sf_1_dict, epho_bins, cos_bins)

###### Butanol Free-proton Peak Parameter Extraction (MUST)

In [ ]:
#Butanol - SF*Carbon Parameter Exraction
def buta_free_params(buta_dict={}, carb_dict={}, sf_dict={}, epho_bins=[], cos_bins={}):
    #=====================================================================================
    #Dictionary to be returned
    bf_params_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        #list: For each epho bin, containing a list for each angle bin
        bf_1epho_cos_list = []
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Corresponding Scale factor for the interested bin
            sf = sf_dict[epho_keys[epho_bin]][cos_bin]

            #=====================================================================================
            #Determine Bin sizes
            bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

            #Make histograms first
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
            hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)

            #=====================================================================================
            #If SF is too large that hist_bf have negative value, change sf and compute again
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb
            counter_bad_bin  = 0 
            for i in range(len(hist_bf)):
                if (hist_bf[i] <= 0):
                    counter_bad_bin += 1
            #If # of bins that have larger sf*carb than butanol is more than 1/20 of butanol bins, half the sf
            if (counter_bad_bin >= int(len(bins_bf)/10)):
                #CHANGE the DENOMENATOR if too manyless bins of hist_bf have negative values!!!!
                sf = sf/2.5
                                
            #=====================================================================================
            #Free butanol peak histograms
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb

            #If any bin in hist_bf is negative, set it to 0
            for i in range(len(hist_bf)):
                if(hist_bf[i] < 0):
                    hist_bf[i] = 0
            
            #Mids for plotting scale carbon HISTOGRAM
            mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

            #Stats for free butanol histogram
            peak_bf  = bins_buta[np.argmax(hist_buta)]
            mean_bf  = np.average(mids, weights=hist_bf)
            var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
            stddv_bf = np.sqrt(var_bf) 
            max_bf   = np.max(hist_bf)

            #=====================================================================================
            #Shorten the fit range
            #    -peak_bf +/- certain amount of range to enclose the peak and 1 background.
            #    -If the fit range is too wide, the background consists of many gaussians, hindering bg fits.
            xmin_fit = peak_bf - 2*stddv_bf
            xmax_fit = peak_bf + 2*stddv_bf
                        
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
            mids_fit          = mids[boolean_arry_bins]
            
            #subset of hist_bf for fitting
            hist_bf_fit = hist_bf[boolean_arry_bins]
            
            #Stats for free butanol histogram
            peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
            mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
            var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
            stddv_bf_fit = np.sqrt(var_bf_fit) 
            max_bf_fit   = np.max(hist_bf_fit)     
            pedestal     = max_bf_fit/20
            
            #=====================================================================================
            #Fit Function (inner)
            def fit_func_combined(x, A, sigma, B, sigma2):
                #Background function
                #Set mu fixed to mean_bf to reduce dof
                gaussian  = A*np.exp(-1.0*(x - mean_bf)**2  / (2*sigma**2))
                #Peak function
                #Set mu fixed to peak_bf to reduce dof
                gaussian2 = B*np.exp(-1.0*(x - peak_bf)**2 / (2*sigma2**2))
                #Final Func
                final_func = gaussian + gaussian2 + pedestal
                return final_func    

            #=====================================================================================
            #Initial guess
            init_guess = [round(max_bf_fit/2, 3), round(stddv_bf_fit, 3), \
                          round(max_bf_fit, 3), round(stddv_bf_fit/5, 3)]
            
            #Fit
            try:
                params, pconv = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, \
                                          p0=init_guess)

            except RuntimeError:
                #Set empty list
                params  = [np.nan for i in range(len(init_guess))]
                pconv   = [np.nan for i in range(len(init_guess))]                
            
            #=====================================================================================
            #Butanol free peak start/end positions
            buta_start_end = (peak_bf - 3*params[3], peak_bf + 3*params[3])

            #=====================================================================================
            #Append mids, bins_bf, hist_bf, and final params for each cos bins to bf_1epho_cos_lisabs
            bf_1epho_cos_list.append([mids, bins_bf, hist_bf, params, mean_bf, peak_bf, buta_start_end, \
                                      mids_fit, pedestal])
    
        #=========================================================================================
        bf_params_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = bf_1epho_cos_list
        
    return bf_params_dict, epho_bins, cos_bins    

In [ ]:
#bf_params_dist has Bins, x_coordinates, hist_bf, and fitting parameters for each epho/cos_bins 
bf_params_dict, epho_bins, cos_bins = buta_free_params(buta_dict, carb_dict, init_sf_2_dict, \
                                                       epho_bins, cos_bins)

###### Butanol Free-proton Peak Parameter Extraction - Individually (MUST)

In [ ]:
#Fit manually to the specific bins
def manu_fit(epho_bin=0, cos_bin=0, bf_params_dict={}, buta_dict={}, carb_dict={}, init_sf_2_dict={}, \
             fit_rng=1.0, pedestal_fit=20, mean_1=0, mean_2=0, \
             guess_int=[], bounds=([0, 0.0, 0, 0], [np.inf, np.inf, np.inf, np.inf])):
    
    #=====================================================================================
    #Keys for epho bins
    epho_keys = sorted(carb_dict.keys())
    
    #=====================================================================================
    #df declare for butanol and carbon
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

    #Corresponding Scale factor for the interested bin
    sf = init_sf_2_dict[epho_keys[epho_bin]][cos_bin]

    #=====================================================================================
    #Determine Bin sizes
    bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

    #Make histograms first
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
    hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)

    #=====================================================================================
    #Free butanol peak histograms
    #    -If SF is too large that hist_bf have negative value, change sf and compute again
    hist_bf, bins_bf     = hist_buta - sf*hist_carb, bins_carb
    counter_bad_bin      = 0 
    for i in range(len(hist_bf)):
        if (hist_bf[i] <= 0):
            counter_bad_bin += 1
    #If # of bins that have larger sf*carb than butanol is more than 1/10 of butanol bins, half the sf
    if (counter_bad_bin >= int(len(bins_bf)/10)):
        sf = sf/2
    hist_bf, bins_bf     = hist_buta - sf*hist_carb, bins_carb
    
    #If any bin in hist_bf is negative, set it to 0
    for i in range(len(hist_bf)):
        if(hist_bf[i] < 0):
            hist_bf[i] = 0

    #Mids for plotting scale carbon HISTOGRAM
    mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

    #Stats for free butanol histogram
    peak_bf  = bins_buta[np.argmax(hist_buta)]
    mean_bf  = np.average(mids, weights=hist_bf)
    var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
    stddv_bf = np.sqrt(var_bf) 
    max_bf   = np.max(hist_bf)

    #=====================================================================================
    #=====================================================================================
    # Fit Manually 
    #=====================================================================================
    #=====================================================================================
    #Shorten the fit range
    xmin_fit = peak_bf - fit_rng*stddv_bf
    xmax_fit = peak_bf + fit_rng*stddv_bf

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
    mids_fit          = mids[boolean_arry_bins]

    #subset of hist_bf for fitting
    hist_bf_fit = hist_bf[boolean_arry_bins]
    
    #Stats for free butanol histogram
    peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
    mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
    var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
    stddv_bf_fit = np.sqrt(var_bf_fit) 
    max_bf_fit   = np.max(hist_bf_fit) 

    #=====================================================================================
    #Fit Function
    """
    -2 gaussians + pededtals + fixed means + pedestal

    -What I can vary
        -fixed means
        -reduce doe: pedestal, mean, sigma
        -initial guesses
        -bounds
        -fit range
    """
    #Manually varied fitting parameters
    pedestal = max_bf_fit / pedestal_fit
    #pedestal_fit = max_bf_fit/20
    #mean_1       = 0.032
    #mean_2       = 0.032

    #Fit Function (inner)
    def fit_func_combined(x, A, sigma, B, sigma2):
        #Background function
        #Set mu fixed to mean_bf to reduce dof
        gaussian  = A*np.exp(-1.0*(x - mean_1)**2  / (2*sigma**2))
        #Peak function
        #Set mu fixed to peak_bf to reduce dof
        gaussian2 = B*np.exp(-1.0*(x - mean_2)**2 / (2*sigma2**2))
        #Final Func
        final_func = gaussian + gaussian2 + pedestal
        return final_func   

    #=====================================================================================
    #Initial guesses for the parameters to fit:
    #    -3 amplitudes, means and standard deviations plus a pedestal.
    guess = [max_bf_fit/guess_int[0], stddv_bf_fit/guess_int[1], \
             max_bf_fit/guess_int[2],   stddv_bf_fit/guess_int[3]]

    #Bounds for parameters
    #bounds = ([0, 0.0, 0, 0], \
    #          [np.inf, np.inf, np.inf, np.inf])

    #Fit
    params, pcov = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, p0=guess, bounds=bounds)

    #=====================================================================================
    #Butanol free peak start/end positions
    mids_fit_numpy = np.array(mids_fit)
    final_peak_loc = mids_fit_numpy[np.argmax(fit_func_combined(mids_fit_numpy, *params))]

    #Store Butanol Start/end position
    buta_start_end = (final_peak_loc - 3*params[3], final_peak_loc + 3*params[3])    
    
    #Store redone fit parameters
    bf_params_dict[epho_keys[epho_bin]][cos_bin][0] = mids
    bf_params_dict[epho_keys[epho_bin]][cos_bin][3] = params
    bf_params_dict[epho_keys[epho_bin]][cos_bin][4] = mean_1
    bf_params_dict[epho_keys[epho_bin]][cos_bin][5] = mean_2
    bf_params_dict[epho_keys[epho_bin]][cos_bin][6] = buta_start_end
    bf_params_dict[epho_keys[epho_bin]][cos_bin][7] = mids_fit
    bf_params_dict[epho_keys[epho_bin]][cos_bin][8] = pedestal

In [ ]:
#Plot refits
def plot_fixed(epho_bin=0, cos_bin=0, bf_params_dict={}, epho_bins=[], cos_bins={}):

    #Plot size
    fit_redo = plt.figure(figsize=(15,10))
    
    #Keys for epho
    epho_keys = sorted(bf_params_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #Fit function
    def fit_func_combined(x, A, mu, sigma, B, mu2, sigma2, pedestal):
        #Background function
        gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
        #Peak function
        gaussian2 = B*np.exp(-1.0*(x - mu2)**2 / (2*sigma2**2))
        #Pedestal
        pedestal  = pedestal
        #Final Func
        final_func = gaussian + gaussian2 + pedestal 
        return final_func

    #1 fit fucnt
    def fit_func_1(x, A, mu, sigma, pedestal):
        #Background function
        gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
        #Final Func
        final_func = gaussian + pedestal 
        return final_func
    
    #Extracted fit parameters
    A_fit              = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][0]
    sigma_fit          = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][1]
    B_fit              = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][2]
    sigma2_fit         = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][3]
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    pedestal           = bf_params_dict[epho_keys[epho_bin]][cos_bin][8]
    mu_fit             = bf_params_dict[epho_keys[epho_bin]][cos_bin][4]
    mu2_fit            = bf_params_dict[epho_keys[epho_bin]][cos_bin][5]

    #Bins, x_coordinates, hist_bf
    mids        = np.array(bf_params_dict[epho_keys[epho_bin]][cos_bin][0]) 
    mids_fit    = np.array(bf_params_dict[epho_keys[epho_bin]][cos_bin][7]) 
    bins_bf     = bf_params_dict[epho_keys[epho_bin]][cos_bin][1]
    hist_bf     = bf_params_dict[epho_keys[epho_bin]][cos_bin][2]

    #Examine each fit funcations separately
    y_val       = fit_func_combined(mids_fit, \
                                    A_fit, mu_fit, sigma_fit, \
                                    B_fit, mu2_fit, sigma2_fit, \
                                    pedestal)
    y_val_1 = fit_func_1(mids_fit, A_fit, mu_fit, sigma_fit, pedestal)
    y_val_2 = fit_func_1(mids_fit, B_fit, mu2_fit, sigma2_fit, pedestal)

    #Plot each fit function separately
    plt.plot(mids_fit, y_val_1, linewidth=2, label="fit_func_1")
    plt.plot(mids_fit, y_val_2, linewidth=2, label="fit_func_2")

    #Plot butanol free histogram
    plt.bar(mids, hist_bf, width=bins_bf[1]-bins_bf[0], alpha=0.7, label="buta-sf*carb", color="#4C72B0")

    #Plot fitting 
    plt.plot(mids_fit, y_val, linewidth=2, label="Buta free-nucl", color="#C44E52")

    #Butanol free nucleon sharp peak start/end positions
    plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Texts
    plt.text(0.03, 0.53, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
             .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
             fontsize=22, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Labels, ticks, lim
    plt.xlabel("Missing mass sq. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    # plt.xlim(-0.3, 0.3)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=20, loc="upper left")

    plt.show()
    fit_redo.savefig("./run_period_1/fit_redo_{:d}_{:d}".format(epho_bin, cos_bin))    

In [ ]:
#COS=0


epho_bin, cos_bin = 0, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, 0.025, np.inf, 0.005]))

epho_bin, cos_bin = 1, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 2], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

epho_bin, cos_bin = 2, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.032, mean_2=0.032, \
                                                           guess_int=[5, 1, 1, 2], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

epho_bin, cos_bin = 3, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.03, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

epho_bin, cos_bin = 4, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.03, \
                                                           guess_int=[5, 2, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

epho_bin, cos_bin = 5, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.03, \
                                                           guess_int=[5, 2, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

epho_bin, cos_bin = 6, 0
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=0.5, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.03, \
                                                           guess_int=[5, 2, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))


#COS=1

epho_bin, cos_bin = 0, 1
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, 0.025, np.inf, 0.005]))
epho_bin, cos_bin = 1, 1
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 2], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 2, 1
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.032, mean_2=0.0325, \
                                                           guess_int=[5, 1, 1, 2], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 3, 1
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.0325, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 4, 1
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.0325, \
                                                           guess_int=[5, 2, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 5, 1
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.03, \
                                                           guess_int=[5, 2, 1, 15], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, 0.005]))

#COS=2

epho_bin, cos_bin = 0, 2
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, 0.025, np.inf, 0.005]))
epho_bin, cos_bin = 1, 2
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 2], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 2, 2
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.032, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 2], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 3, 2
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.025, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

#COS=3

epho_bin, cos_bin = 0, 3
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1, pedestal_fit=10, \
                                                           mean_1=0.001, mean_2=0.0325, \
                                                           guess_int=[5, 1.5, 1, 100], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 2, 3
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=30, \
                                                           mean_1=0.002, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 30], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 3, 3
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=30, \
                                                           mean_1=0.002, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 30], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))


#COS=4

epho_bin, cos_bin = 2, 4
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1, pedestal_fit=10, \
                                                           mean_1=0.001, mean_2=0.0325, \
                                                           guess_int=[5, 1.5, 1, 100], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 3, 4
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=30, \
                                                           mean_1=0.002, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 30], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 4, 4
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=30, \
                                                           mean_1=0.002, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 30], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
# epho_bin, cos_bin = 5, 4
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=30, \
                                                           mean_1=0.002, mean_2=0.035, \
                                                           guess_int=[5, 1, 1, 30], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

#COS=5

epho_bin, cos_bin = 0, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.06, mean_2=0.04, \
                                                           guess_int=[2, 12, 1, 2], 
                                                           bounds=([0,      0.0,   0,      0], \
                                                                   [np.inf, 0.085, np.inf, np.inf]))
epho_bin, cos_bin = 1, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.06, mean_2=0.04, \
                                                           guess_int=[2, 12, 1, 2], 
                                                           bounds=([0,      0.0,   0,      0], \
                                                                   [np.inf, 0.085, np.inf, np.inf]))
epho_bin, cos_bin = 2, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.03, mean_2=0.04, \
                                                           guess_int=[2, 0.4, 1, 100], 
                                                           bounds=([0,      0.05,   0,      0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 3, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.03, mean_2=0.0375, \
                                                           guess_int=[1, 1, 1, 10], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, 0.01]))
epho_bin, cos_bin = 4, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.03, mean_2=0.04, \
                                                           guess_int=[5, 0.5, 1, 10], 
                                                           bounds=([0,      0.04,   0,      0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 5, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 6, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 7, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 8, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, 0.07, np.inf, np.inf]))
epho_bin, cos_bin = 17, 5
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=0.75, pedestal_fit=20, \
                                                           mean_1=0.125, mean_2=0.0375, \
                                                           guess_int=[5, 10, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

#COS=6


epho_bin, cos_bin = 3, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.03, mean_2=0.04, \
                                                           guess_int=[1, 1, 1, 10], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, 0.01]))
epho_bin, cos_bin = 4, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.05, mean_2=0.0425, \
                                                           guess_int=[5, 0.5, 1, 10], 
                                                           bounds=([0,      0.04,   0,      0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 5, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 6, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 7, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 8, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.02, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 9, 6
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.02, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

#COS=7


epho_bin, cos_bin = 2, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.03, mean_2=0.04, \
                                                           guess_int=[1, 1, 1, 10], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, 0.01]))
epho_bin, cos_bin = 3, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.03, mean_2=0.04, \
                                                           guess_int=[1, 1, 1, 10], 
                                                           bounds=([0, 0.0, 0, 0], \
                                                                   [np.inf, np.inf, np.inf, 0.01]))
epho_bin, cos_bin = 4, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.05, mean_2=0.0425, \
                                                           guess_int=[5, 0.5, 1, 10], 
                                                           bounds=([0,      0.04,   0,      0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 5, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 6, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 7, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=1.0, pedestal_fit=20, \
                                                           mean_1=-0.015, mean_2=0.04, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([     0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 8, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.02, mean_2=0.035, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 9, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.02, mean_2=0.0375, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 10, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.02, mean_2=0.0375, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 11, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.6, pedestal_fit=20, \
                                                           mean_1=0.03, mean_2=0.0375, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 12, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.08, mean_2=0.0375, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))
epho_bin, cos_bin = 13, 7
#Call mannu_fit
manu_fit(epho_bin, cos_bin, bf_params_dict, buta_dict, carb_dict, init_sf_2_dict, \
                                                           fit_rng=.5, pedestal_fit=20, \
                                                           mean_1=0.08, mean_2=0.0375, \
                                                           guess_int=[5, 1.5, 1, 10], 
                                                           bounds=([0,   0.0,      0,     0], \
                                                                   [np.inf, np.inf, np.inf, np.inf]))

###### BUTA fit + CARB fit: Separately - GLOBAL (MUST) 

In [ ]:
#Fit Buta:
def fit_buta_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                    epho_bins=[], cos_bins={}, \
                    left_fit_range=2.5, right_fit_range=2.5, \
                    bin_size_mltp = 0.5, \
#                     buta_init_guess_mltplr = [1, 0.5, 1, 1, 1, 1, 1], \
#                     bounds_buta_mltplr = ([0.5, 0.1, 0, -np.inf, -np.inf, -np.inf, -np.inf], \
#                                           [1.5, 1, np.inf, np.inf, np.inf, np.inf, np.inf])):
                    buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                    bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1])):
    
    #Dict to store final
    buta_fit_dict = {}
    
    #Dict to store err msgs
    err_dict = {}
    cnt      = 0    
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        buta_1epho_cos_dict = {}
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            peak_buta   = buta_bins[np.argmax(hist_buta)]
            mean_buta   = np.average(bin_centers, weights=hist_buta)
            var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
            stddev_buta = np.sqrt(var_buta)
            num_buta    = len(butanol_data)
            max_buta    = np.max(hist_buta)
            
            #=================================================================================================
            #Fitting range
            #left_fit_range  = 2.5
            #right_fit_range = 2.5
            start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                         (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

            #Prevent fits to be delcared out of carbon dist. range
            carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            if (start_end[0] < carb_lims[0]):
                start_end[0] = carb_lims[0]
            if (start_end[1] > carb_lims[1]):
                start_end[1] = carb_lims[1]
            
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            #    -this histogram will be used for final chisq minimization fitting!!!
            hist_buta_new = hist_buta[boolean_arry_bins]

            #Stats BUTA new
            mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
            var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
            stddev_buta_new = np.sqrt(var_buta_new)
            max_buta_new    = np.max(hist_buta_new)    
            sigma_buta      = np.sqrt(hist_buta_new)
        
            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_buta)):
                if (sigma_buta[i] == 0):
                    sigma_buta[i] = 0.01
        
            #=================================================================================================
            #Fit buta
            def fit_buta(x, A, sigma1, B, mu, sigma2, a):
                gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus1 + gaus2 + poly
                return y
            
            #Bounds: needed variables
            bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                                      bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])         
            
            #initial guess + bounds
            buta_init_guess = [buta_init_guess_mltplr[0]*max_buta_new, \
                               buta_init_guess_mltplr[1]*stddev_buta_new, \
                               buta_init_guess_mltplr[2]*max_buta_new, \
                               buta_init_guess_mltplr[3]*peak_buta, \
                               buta_init_guess_mltplr[4]*stddev_buta_new, \
                               buta_init_guess_mltplr[5]*max_buta_new]        
            #Bounds of carb
            bounds_buta = ([bounds_buta_mltplr[0][0]*max_buta_new, bounds_buta_mltplr[0][1]*stddev_buta_new, \
                            bounds_buta_mltplr[0][2]*max_buta_new, bounds_buta_mltplr[0][3]*bf_strt_end_dist + peak_buta, 
                            bounds_buta_mltplr[0][4]*stddev_buta_new, \
                            bounds_buta_mltplr[0][5]*max_buta_new]
                           , \
                           [bounds_buta_mltplr[1][0]*max_buta_new, bounds_buta_mltplr[1][1]*stddev_buta_new, \
                            bounds_buta_mltplr[1][2]*max_buta_new, bounds_buta_mltplr[1][3]*bf_strt_end_dist + peak_buta, 
                            bounds_buta_mltplr[1][4]*stddev_buta_new, \
                            bounds_buta_mltplr[1][5]*max_buta_new])                          

            #Curve_fit
            try:
                buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
                                                                 ydata=hist_buta_new, \
                                                                 p0=buta_init_guess, bounds=bounds_buta, \
                                                                 sigma=sigma_buta, absolute_sigma=True)
            except RuntimeError as err1:
                cnt += 1
                err_dict["{:d}".format(cnt)] = [epho_bin, cos_bin, epho_bins[epho_bin], \
                                                cos_bins[cos_keys[epho_bin]][cos_bin], err1]
                #Set empty list
                buta_popt  = [0 for i in range(len(buta_init_guess))]
                buta_pconv = [0 for i in range(len(buta_init_guess))]
                    
            #=================================================================================================
            #Into dicts
            buta_1epho_1cos_dict = {"bin_centers_new":bin_centers_new, "buta_bins":buta_bins, \
                                   "num_buta":num_buta, "peak_buta":peak_buta, \
                                   "buta_popt":buta_popt, "start_end":start_end, \
                                   "bin_size_mltp":bin_size_mltp, "buta_pconv":buta_pconv} 

            #Append
            buta_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = buta_1epho_1cos_dict

        #=================================================================================================
        #Save to dictionary
        buta_fit_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = buta_1epho_cos_dict
        
    return buta_fit_dict, err_dict, epho_bins, cos_bins

In [ ]:
#Call fit_buta
buta_fit_dict, buta_err_dict, epho_bins, cos_bins = \
fit_buta_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                bf_params_dict=bf_params_dict, \
                init_sf_2_dict=init_sf_2_dict, \
                epho_bins=epho_bins, cos_bins=cos_bins, \
                left_fit_range=2.5, right_fit_range=2.5, \
#                 buta_init_guess_mltplr = [1, 0.5, 1, 1, 1, 1, 1], \
#                 bounds_buta_mltplr = ([0.5, 0.1, 0, 0, 0, 0, 0], \
#                                       [1.5, 1, 0.2, np.inf, np.inf, np.inf, np.inf]))
                buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1]))

In [ ]:
#Fit Carb:
def fit_carb_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                    epho_bins=[], cos_bins={}, \
                    left_fit_range=2.5, right_fit_range=2.5, \
                    bin_size_mltp=0.5, \
                    carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 1], \
                    bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 10, 1, 2, 10, np.inf])):
    
    #Dict to store final
    carb_fit_dict = {}
    
    #Dict to store err msgs
    err_dict = {}
    cnt      = 0
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        carb_1epho_cos_dict = {}
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            peak_buta   = buta_bins[np.argmax(hist_buta)]
            mean_buta   = np.average(bin_centers, weights=hist_buta)
            var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
            stddev_buta = np.sqrt(var_buta)
            num_buta    = len(butanol_data)
            max_buta    = np.max(hist_buta)
            #Stats CARB
            peak_carb   = buta_bins[np.argmax(hist_carb)]
            mean_carb   = np.average(bin_centers, weights=hist_carb)
            var_carb    = np.average((bin_centers-mean_carb)**2, weights=hist_carb)
            stddev_carb = np.sqrt(var_carb)
            num_carb    = len(carbon_data)
            max_carb    = np.max(hist_carb)
            
            #=================================================================================================
            #Fitting range
            #left_fit_range  = 2.5
            #right_fit_range = 2.5
            start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                         (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

            #Prevent fits to be delcared out of carbon dist. range
            carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            if (start_end[0] < carb_lims[0]):
                start_end[0] = carb_lims[0]
            if (start_end[1] > carb_lims[1]):
                start_end[1] = carb_lims[1]
            
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            hist_carb_new = hist_carb[boolean_arry_bins]

            #Stats CARB new
            mean_carb_new   = np.average(bin_centers_new, weights=hist_carb_new)
            var_carb_new    = np.average((bin_centers_new-mean_carb_new)**2, weights=hist_carb_new)
            stddev_carb_new = np.sqrt(var_carb_new)
            max_carb_new    = np.max(hist_carb_new)
            sigma_carb      = np.sqrt(hist_carb_new)

            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_carb)):
                if (sigma_carb[i] == 0):
                    sigma_carb[i] = 0.01            
            
            #=================================================================================================
            #Carbon fit
            def fit_carb(x, A, sigma1, B, mu, sigma2, a):
                #remaining ice
                gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
                gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly       = a
                y          = gaussian + gaussian_2 + poly
                return y

            #Bounds: needed variables
            bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                                      bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])

            
            #initial guess + bounds
            carb_init_guess = [carb_init_guess_mltplr[0]*max_carb_new, \
                               carb_init_guess_mltplr[1]*stddev_carb_new, \
                               carb_init_guess_mltplr[2]*max_carb_new, \
                               carb_init_guess_mltplr[3]*peak_carb, \
                               carb_init_guess_mltplr[4]*stddev_carb_new, \
                               carb_init_guess_mltplr[5]*max_carb_new]      
            #Bounds of carb
            bounds_carb = ([bounds_carb_mltplr[0][0]*max_carb_new, bounds_carb_mltplr[0][1]*stddev_carb_new, \
                            bounds_carb_mltplr[0][2]*max_carb_new, bounds_carb_mltplr[0][3]*bf_strt_end_dist + peak_carb, 
                            bounds_carb_mltplr[0][4]*stddev_carb_new, \
                            bounds_carb_mltplr[0][5]*max_carb_new]
                           , \
                           [bounds_carb_mltplr[1][0]*max_carb_new, bounds_carb_mltplr[1][1]*stddev_carb_new, \
                            bounds_carb_mltplr[1][2]*max_carb_new, bounds_carb_mltplr[1][3]*bf_strt_end_dist + peak_carb, 
                            bounds_carb_mltplr[1][4]*stddev_carb_new, \
                            bounds_carb_mltplr[1][5]*max_carb_new])              
            
            try:
                #Curve_Fit carbon
                carb_popt, carb_pconv = scipy.optimize.curve_fit(fit_carb, xdata=bin_centers_new, ydata=hist_carb_new, \
                                                                 p0=carb_init_guess, bounds=bounds_carb, \
                                                                 sigma=sigma_carb, absolute_sigma=True)
            
            except RuntimeError as err1:
                cnt += 1
                err_dict["{:d}".format(cnt)] = [epho_bin, cos_bin, epho_bins[epho_bin], \
                                                cos_bins[cos_keys[epho_bin]][cos_bin], err1]
                #Set empty list
                carb_popt  = [0 for i in range(len(carb_init_guess))]
                carb_pconv = [0 for i in range(len(carb_init_guess))]
                
            #=================================================================================================
            #Into dicts
            carb_1epho_1cos_dict = {"bin_centers_new":bin_centers_new, "carb_bins":carb_bins, \
                                   "num_carb":num_carb, "peak_carb":peak_carb, \
                                   "carb_popt":carb_popt, "start_end":start_end, \
                                   "bin_size_mltp":bin_size_mltp, "carb_pconv":carb_pconv} 

            #Append
            carb_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = carb_1epho_1cos_dict

        #=================================================================================================
        #Save to dictionary
        carb_fit_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = carb_1epho_cos_dict
        
    return carb_fit_dict, err_dict, epho_bins, cos_bins

In [ ]:
#Call fit_carb
carb_fit_dict, carb_err_dict, epho_bins, cos_bins = \
fit_carb_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                bf_params_dict=bf_params_dict, \
                init_sf_2_dict=init_sf_2_dict, \
                epho_bins=epho_bins, cos_bins=cos_bins, \
                left_fit_range=2.5, right_fit_range=2.5, \
                carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
                bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, 1]))

###### BUTA fit + CARB fit: Separately - INDIVIDUAL (MUST) 

In [ ]:
#Buta fit individual
def fit_buta_indi(epho_bin=0, cos_bin=0, epho_bins=[], cos_bins={}, \
                  buta_dict={}, carb_dict={}, \
                  bf_params_dict={}, init_sf_2_dict={}, \
                  left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1, \
                  buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                  bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1]), \
                  new_peak=0):
    
    #keys for epho_bins
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]
    
    #=================================================================================================
    #Into numpy
    butanol_data  = np.array(df_buta)
    carbon_data   = np.array(df_carb)
    #bin_size_mltp = 1

    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    peak_buta   = buta_bins[np.argmax(hist_buta)]
    mean_buta   = np.average(bin_centers, weights=hist_buta)
    var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    num_buta    = len(butanol_data)
    max_buta    = np.max(hist_buta)    
    
    #New peak
    if (new_peak != 0):
        peak_buta = new_peak
    
    #=================================================================================================
    #Fitting range
    #left_fit_range  = 2.5
    #right_fit_range = 2.5
    start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                 (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

    #Prevent fits to be delcared out of carbon dist. range
    carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    if (start_end[0] < carb_lims[0]):
        start_end[0] = carb_lims[0]
    if (start_end[1] > carb_lims[1]):
        start_end[1] = carb_lims[1]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    #    -this histogram will be used for final chisq minimization fitting!!!
    hist_buta_new = hist_buta[boolean_arry_bins]

    #Stats BUTA new
    mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
    var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
    stddev_buta_new = np.sqrt(var_buta_new)
    max_buta_new    = np.max(hist_buta_new)        
    sigma_buta      = np.sqrt(hist_buta_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_buta)):
        if (sigma_buta[i] == 0):
            sigma_buta[i] = 0.01
            
    #=================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Bounds: needed variables
    bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                              bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])         

    #initial guess + bounds
    buta_init_guess = [buta_init_guess_mltplr[0]*max_buta_new, \
                       buta_init_guess_mltplr[1]*stddev_buta_new, \
                       buta_init_guess_mltplr[2]*max_buta_new, \
                       buta_init_guess_mltplr[3]*peak_buta, \
                       buta_init_guess_mltplr[4]*stddev_buta_new, \
                       buta_init_guess_mltplr[5]*max_buta_new]        
    #Bounds of carb
    bounds_buta = ([bounds_buta_mltplr[0][0]*max_buta_new, bounds_buta_mltplr[0][1]*stddev_buta_new, \
                    bounds_buta_mltplr[0][2]*max_buta_new, bounds_buta_mltplr[0][3]*bf_strt_end_dist + peak_buta, 
                    bounds_buta_mltplr[0][4]*stddev_buta_new, \
                    bounds_buta_mltplr[0][5]*max_buta_new]
                   , \
                   [bounds_buta_mltplr[1][0]*max_buta_new, bounds_buta_mltplr[1][1]*stddev_buta_new, \
                    bounds_buta_mltplr[1][2]*max_buta_new, bounds_buta_mltplr[1][3]*bf_strt_end_dist + peak_buta, 
                    bounds_buta_mltplr[1][4]*stddev_buta_new, \
                    bounds_buta_mltplr[1][5]*max_buta_new])

    #Curve_Fit
#     buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
#                                                      ydata=hist_buta_new, \
#                                                      p0=buta_init_guess, bounds=bounds_buta)
    
    buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
                                                     ydata=hist_buta_new, \
                                                     p0=buta_init_guess, bounds=bounds_buta, \
                                                     sigma=sigma_buta, absolute_sigma=True)

    
    #Store redone fits into buta_fit_dict
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"] = bin_centers_new
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]       = buta_bins
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["num_buta"]        = num_buta
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]       = peak_buta
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]       = buta_popt
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_pconv"]      = buta_pconv
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]       = start_end
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]   = bin_size_mltp

In [ ]:
#Buta fit individual
def fit_carb_indi(epho_bin=0, cos_bin=0, epho_bins=[], cos_bins={}, \
                  buta_dict={}, carb_dict={}, \
                  bf_params_dict={}, init_sf_2_dict={}, \
                  left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1, \
                  carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 1], \
                  bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, np.inf]), \
                  new_peak=0):
    
    #keys for epho_bins
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())
    
    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #=================================================================================================
    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]
    
    #=================================================================================================
    #Into numpy
    butanol_data  = np.array(df_buta)
    carbon_data   = np.array(df_carb)
    #bin_size_mltp = 1
    
    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    peak_buta   = buta_bins[np.argmax(hist_buta)]
    mean_buta   = np.average(bin_centers, weights=hist_buta)
    var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    num_buta    = len(butanol_data)
    max_buta    = np.max(hist_buta)    
    #Stats CARB
    peak_carb   = buta_bins[np.argmax(hist_carb)]
    mean_carb   = np.average(bin_centers, weights=hist_carb)
    var_carb    = np.average((bin_centers-mean_carb)**2, weights=hist_carb)
    stddev_carb = np.sqrt(var_carb)
    num_carb    = len(carbon_data)
    max_carb    = np.max(hist_carb) 
    
    #New peak
    if (new_peak != 0):
        peak_buta = new_peak
        peak_carb = new_peak
    
    #=================================================================================================
    #Fitting range
    #left_fit_range  = 2.5
    #right_fit_range = 2.5
    start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                 (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

    #Prevent fits to be delcared out of carbon dist. range
    carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    if (start_end[0] < carb_lims[0]):
        start_end[0] = carb_lims[0]
    if (start_end[1] > carb_lims[1]):
        start_end[1] = carb_lims[1]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    hist_carb_new = hist_carb[boolean_arry_bins]

    #Stats CARB new
    mean_carb_new   = np.average(bin_centers_new, weights=hist_carb_new)
    var_carb_new    = np.average((bin_centers_new-mean_carb_new)**2, weights=hist_carb_new)
    stddev_carb_new = np.sqrt(var_carb_new)
    max_carb_new    = np.max(hist_carb_new)       
    sigma_carb      = np.sqrt(hist_carb_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_carb)):
        if (sigma_carb[i] == 0):
            sigma_carb[i] = 0.01
    
    #=================================================================================================
    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Bounds: needed variables
    bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                              bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])


    #initial guess + bounds
    carb_init_guess = [carb_init_guess_mltplr[0]*max_carb_new, \
                       carb_init_guess_mltplr[1]*stddev_carb_new, \
                       carb_init_guess_mltplr[2]*max_carb_new, \
                       carb_init_guess_mltplr[3]*peak_carb, \
                       carb_init_guess_mltplr[4]*stddev_carb_new, \
                       carb_init_guess_mltplr[5]*max_carb_new]        
    #Bounds of carb
    bounds_carb = ([bounds_carb_mltplr[0][0]*max_carb_new, bounds_carb_mltplr[0][1]*stddev_carb_new, \
                    bounds_carb_mltplr[0][2]*max_carb_new, bounds_carb_mltplr[0][3]*bf_strt_end_dist + peak_carb, 
                    bounds_carb_mltplr[0][4]*stddev_carb_new, \
                    bounds_carb_mltplr[0][5]*max_carb_new]
                   , \
                   [bounds_carb_mltplr[1][0]*max_carb_new, bounds_carb_mltplr[1][1]*stddev_carb_new, \
                    bounds_carb_mltplr[1][2]*max_carb_new, bounds_carb_mltplr[1][3]*bf_strt_end_dist + peak_carb, 
                    bounds_carb_mltplr[1][4]*stddev_carb_new, \
                    bounds_carb_mltplr[1][5]*max_carb_new])  

    #Curve_Fit
    carb_popt, carb_pconv = scipy.optimize.curve_fit(fit_carb, xdata=bin_centers_new, \
                                                     ydata=hist_carb_new, \
                                                     p0=carb_init_guess, bounds=bounds_carb, \
                                                     sigma=sigma_carb, absolute_sigma=True)

    #Store redone fits into carb_fit_dict
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"] = bin_centers_new
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_bins"]       = carb_bins
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["num_carb"]        = num_carb
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]       = peak_carb
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]       = carb_popt
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_pconv"]      = carb_pconv
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]       = start_end
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]   = bin_size_mltp    

In [ ]:
#Indi fit plot
def plot_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, epho_bins=[], cos_bins={}, \
              buta_fit_dict={}, carb_fit_dict={}, buta_dict={}, carb_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    fit_redo = plt.figure(figsize=(10,6))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]        
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=0.9, range=(range_list[0], range_list[1]), \
             color="#55A868")
    
    #Plot fitting
    plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
             color="red", alpha=0.7, ls="-", label="buta fit")
    plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
             color="blue", alpha=0.7, ls="-", label="buta bound fit")            
    plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
             color="red", alpha=0.7, ls="--", label="carb fit")
    plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
             color="blue", alpha=0.7, ls="--", label="carb bound_fit")            
    
    ##Butanol free nucleon sharp peak start/end positions
    #plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    #plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #===================================================================================================
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=15)
    plt.ylabel("Counts", fontsize=15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    ##Texts
    #plt.text(0.03, 0.53, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
    #         .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
    #         fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=15)
        
    #Legends
    plt.legend(fontsize=10, loc="upper left")

    plt.show()
    fit_redo.savefig("./run_period_1/sf_fit_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
##COS=0

epho_bin, cos_bin = 0, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7.5, right_fit_range=7.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=5.5, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 1, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]), \
              new_peak=0.033)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 4, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]), \
              new_peak=0.03
              )
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 7, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=5.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.5, 0, 0], [1, 1, 1, 0.5, 1, 1]))


epho_bin, cos_bin = 11, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7.5, right_fit_range=7.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=5.5, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))


epho_bin, cos_bin = 12, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7.5, right_fit_range=7.5, bin_size_mltp=1.2, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=5.5, bin_size_mltp=1.2, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 14, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7.5, right_fit_range=7.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=5.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 17, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=4.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 18, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=2., bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2.5, bin_size_mltp=.75, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

###COS=1

epho_bin, cos_bin = 1, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]), \
              new_peak=0.033)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))


epho_bin, cos_bin = 4, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.03)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=3., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, .5, 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 5, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.03)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, .5, 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 7, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 9, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]), 
              new_peak=.03)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 11, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 12, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 13, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 14, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 15, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 16, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 19, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -2., 0, 0], [1, 1, 1, 2., 10, 1]), \
              new_peak=0.02)

###COS=2

epho_bin, cos_bin = 0, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5., right_fit_range=1.2, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0., 0., 0.2, -0.55, 0.1, 0.1], [1, 0.5, 1, 0.005, 1., 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=2.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 1, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=1.1, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0., 0., 0.2, -.6, 0.1, 0.1], [1, 1, 1, 1.5, 1.5, 0.5]), \
              new_peak=0.033)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 2, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3., bin_size_mltp=1.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.05, 0.1, -1, 0.1, 0], [1, 0.5, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=1.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 7, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=.75, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 11, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3., bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.05, 0.1, -1, 0.1, 0], [1, 0.5, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 13, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=1.25, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.05, 0.1, -1, 0.1, 0], [1, 0.5, 1, 0.15, 1., 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 0.5, 1., 1, 0.5]), \
              new_peak=0.02)

epho_bin, cos_bin = 14, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 15, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 16, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 17, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

# ##COS=3

epho_bin, cos_bin = 0, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1., bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.25, right_fit_range=1.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 1, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=.6, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=1., \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 1, 1, 2, 3, 1]))

epho_bin, cos_bin = 2, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.05, 0.1, -1, 0.1, 0], [1, 0.5, 1, 1, 5, 0.5]), \
              new_peak = 0.0325)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 3, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.05, 0.1, -1, 0.1, 0], [1, 0.5, 1, 1, 5, 0.5]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))

epho_bin, cos_bin = 4, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.05, 0.1, -1, 0.1, 0], [1, 0.5, 1, 1, 5, 0.3]), \
              new_peak=0.0325)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 1, 1, 1, 1, 1]))


epho_bin, cos_bin = 7, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 11, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 14, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 15, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 0.7, 1, 1, 5, 0.3]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=1.25, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -0.25, 0, 0], [1, 1, 1, 0.25, 1, 1]))

epho_bin, cos_bin = 19, 3
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=.25, bin_size_mltp=0.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -2., 0.1, 0], [1, 0.7, 1, 2., 5, 0.3]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.25, right_fit_range=.25, bin_size_mltp=.5, \
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0, 0, 0, -.5, 0, 0], [1, 1, 1, .5, 5, 1]), \
              new_peak=0.03)

##COS=4

epho_bin, cos_bin = 4, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, 1]))

epho_bin, cos_bin = 5, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0.], [1, 10, 1, 2, 10, 1]))

epho_bin, cos_bin = 6, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=3.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, 1]))

epho_bin, cos_bin = 10, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=5.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 3.5, 5, 1.]), \
              new_peak=0.0325)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=4., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 1, 1, 2, 10, 1]), 
              new_peak=0.025)

epho_bin, cos_bin = 11, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=5.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 3.5, 5, 1.]), \
              new_peak=0.034)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=4., bin_size_mltp=1.,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 1, 1, 2, 10, 1]))

epho_bin, cos_bin = 12, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=5.5, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 3.5, 5, 1.]), \
              new_peak=0.034)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=4., bin_size_mltp=1.,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 1, 1, 2, 10, 1]))

epho_bin, cos_bin = 18, 4
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=.25, bin_size_mltp=1., \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -0.5, 1., 0.1], [1, 0.7, 1, 3.5, 5, 1.]), \
              new_peak=0.034)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1., right_fit_range=.25, bin_size_mltp=1.,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 1, 1, 2, 10, 1]), \
              new_peak=0.025)

##COS=5

epho_bin, cos_bin = 1, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4.5, right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=6., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 2, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=6., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 8, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=2.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]))

##COS=6

epho_bin, cos_bin = 1, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=1.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2., bin_size_mltp=.5,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]), \
              new_peak=0.0275)

epho_bin, cos_bin = 2, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1., bin_size_mltp=.5,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 3, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1., bin_size_mltp=.5,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 4, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]))

epho_bin, cos_bin = 5, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]))

epho_bin, cos_bin = 6, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]))

epho_bin, cos_bin = 7, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]))

epho_bin, cos_bin = 10, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]))

##COS=7

epho_bin, cos_bin = 2, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.5, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.4, right_fit_range=2., bin_size_mltp=.5,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 5, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 3, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 8, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 9, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 10, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=5.5, right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 11, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 12, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7.5, right_fit_range=5., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 13, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=4., right_fit_range=3.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]), \
              new_peak=0.0375)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=7.5, right_fit_range=5., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]), \
              new_peak=0.025)

epho_bin, cos_bin = 14, 7
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -5.5, 1., 0.1], [1, 0.7, 1, 0.5, 5, 1.]))
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.3, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 1, 1, 0., 3, 1]))

#COS=8

epho_bin, cos_bin = 5, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.4, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1., right_fit_range=.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 6, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.3, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.75, right_fit_range=1., bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 7, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.75, right_fit_range=.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 8, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 9, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 10, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 11, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

epho_bin, cos_bin = 12, 8
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.5, right_fit_range=.5, bin_size_mltp=.75, \
              buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([0.1, 0.1, 0.1, -.2, 1., 0.1], [1, 0.7, 1, 1.5, 5, 1.]), \
              new_peak=0.04)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=.75, right_fit_range=.75, bin_size_mltp=.75,\
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
              bounds_carb_mltplr = ([0, 0, 0, -0.3, 0, 0], [1, 0.5, 2, 0., 3, 1]))

######  CHISQ min for final SF - GLOBAL (MUST)

In [ ]:
#sf global
def sf_final(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
             epho_bins=[], cos_bins={}, \
             buta_fit_dict={}, carb_fit_dict={}, \
             chi_init_mult=[1], bounds_chi_mult=[0, 2]):

    #Dict to store final sf
    sf_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        sf_1epho_cos_dict = {}
    
        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())
    
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):

            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
        
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_size_mltp = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]    
            bin_num       = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            peak_buta   = buta_bins[np.argmax(hist_buta)]
                       
            #=================================================================================================
            #Fitting range            
            start_end = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]

            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            #    -this histogram will be used for final chisq minimization fitting!!!
            hist_buta_new = hist_buta[boolean_arry_bins]
            hist_carb_new = hist_carb[boolean_arry_bins]

            #Stats BUTA new
            mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
            var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
            stddev_buta_new = np.sqrt(var_buta_new)
            max_buta_new    = np.max(hist_buta_new)    
            sigma_buta      = np.sqrt(hist_buta_new)
            #Stats CARB new
            sigma_carb      = np.sqrt(hist_carb_new)
        
            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_buta)):
                if (sigma_buta[i] == 0):
                    sigma_buta[i] = 0.01
            for i in range(len(sigma_carb)):
                if (sigma_carb[i] == 0):
                    sigma_carb[i] = 0.01

            #=================================================================================================
            #Params from fitting 
            buta_popt = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
            carb_popt = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
                    
            #=================================================================================================
            #Final chisq Fitting
            def chisq(sol, x=np.array([]), \
                      buta_popt=np.array([]), carb_popt=np.array([]), \
                      #sigma_buta=np.array([]), sigma_carb=np.array([]), \
                      hist_buta=np.array([]), hist_carb=np.array([])):
                #Buta bacground
                gaus_buta_back = buta_popt[2]*np.exp(-1.0*(x - buta_popt[3])**2 / (2*(buta_popt[4]**2)))
                poly_buta      = buta_popt[5]
                buta           = gaus_buta_back + poly_buta
                
                #Carb background
                gaus_carb_back = carb_popt[2]*np.exp(-1.0*(x - carb_popt[3])**2 / (2*(carb_popt[4]**2)))
                poly_carb      = carb_popt[5]
                carb           = gaus_carb_back + poly_carb
                
                #each bins
                chisq_list = []
                for i in range(len(x)):
                    #nume
                    nume = pow(buta[i] - sol*carb[i], 2)
                    #deno
                    #deno = pow(sigma_buta[i] + sigma_carb[i], 2)
                    deno = buta[i] + pow(sol, 2)*carb[i]
                    
                    #chisq/bin
                    chisq_bin = nume / deno
                    #chisq_bin = nume

                    #append
                    chisq_list.append(chisq_bin)
                #sum chisq/bin
                chisq = np.sum(chisq_list)
                return chisq            
            
            #chisq initial guess for scale factor
            chi_init_guess = [chi_init_mult[0]*raw_sf]
            
            #chisq bounds for scale factor
            bounds_chi = [(bounds_chi_mult[0]*raw_sf, bounds_chi_mult[1]*raw_sf)]
            
            #Minimize
            popt = scipy.optimize.minimize(chisq, x0=chi_init_guess, bounds=bounds_chi, \
                                           args=(bin_centers_new, buta_popt, carb_popt, \
                                                 hist_buta_new, hist_carb_new), \
                                           method='L-BFGS-B')

            #=================================================================================================
            #Goodness of fit 
            dof       = (len(bin_centers_new) - 1 ) - len(popt.x)
            chisq_per = np.abs(popt.fun) / dof
            
            #=================================================================================================
            #SF Stat uncertainties (NEED WORK)
            sf_stat_err_fit      = 0
            sf_stat_err_chisq    = 0
            
            sf_stat_err = sf_stat_err_fit + sf_stat_err_chisq

            #=================================================================================================
            sf_1epho_1cos_dict = {"popt.x": popt.x[0], "popt.fun":popt.fun, "chisq_per":chisq_per, \
                                  "popt":popt, "sf_stat_err":sf_stat_err}
            
            #Append
            sf_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = sf_1epho_1cos_dict
        
        #=================================================================================================
        #Save to dictionary
        sf_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = sf_1epho_cos_dict
            
    return sf_dict, epho_bins, cos_bins                

In [ ]:
#Discard bad bin
def sf_discard_bad(epho_bins=[], cos_bins={}, sf_dict={}):
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(sf_dict[epho_keys[epho_bin]].keys())
        
        #fn to call within next loop for discarding bins
        def discard_bad_bins(epho_bin=0, cos_bin=0):
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"] = np.nan
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.fun"]  = np.nan
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"] = np.nan

        #=============================================================================================
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
            #epho=0
            if (epho_bin==0 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=1
            elif (epho_bin==1 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=2
            elif (epho_bin==2 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=3
            elif (epho_bin==3 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=4
            elif (epho_bin==4 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=5
            elif (epho_bin==5 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=6
            elif (epho_bin==6 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=7
            elif (epho_bin==7 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=8
            elif (epho_bin==8 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=9
            elif (epho_bin==9 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=10
            elif (epho_bin==10 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=11
            elif (epho_bin==11 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=12
            elif (epho_bin==12 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=13
            elif (epho_bin==13 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==13 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=14
            elif (epho_bin==14 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==14 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=15
            elif (epho_bin==15 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==15 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==15 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=16
            elif (epho_bin==16 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==16 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==16 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==16 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=17
            elif (epho_bin==17 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==17 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==17 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==17 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=18
            elif (epho_bin==18 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=19
            elif (epho_bin==19 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

In [ ]:
#call sf_dict global
sf_dict, epho_bins, cos_bins = sf_final(buta_dict=buta_dict, carb_dict=carb_dict, \
                                        bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
                                        epho_bins=epho_bins, cos_bins=cos_bins, \
                                        buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                                        chi_init_mult=[1], bounds_chi_mult=[0.1, 2.])

#Discard bad bins
sf_discard_bad(epho_bins, cos_bins, sf_dict)

######  CHISQ min for final SF - INDIVIDUAL (MUST)

In [ ]:
#sf_final individual
def sf_final_indi(epho_bin=0, cos_bin=0, \
                  buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                  epho_bins=[], cos_bins={}, \
                  buta_fit_dict={}, carb_fit_dict={}, \
                  chi_init_mult=[1], bounds_chi_mult=[0, 2]):
    
    #keys for epho_bins
    epho_keys      = sorted(carb_dict.keys())
    cos_keys       = sorted(cos_bins.keys())
    cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

    #=================================================================================================
    #Into numpy
    butanol_data = np.array(df_buta)
    carbon_data  = np.array(df_carb)

    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_size_mltp = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]    
    bin_num       = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    peak_buta   = buta_bins[np.argmax(hist_buta)]

    #=================================================================================================
    #Fitting range            
    start_end = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    #    -this histogram will be used for final chisq minimization fitting!!!
    hist_buta_new = hist_buta[boolean_arry_bins]
    hist_carb_new = hist_carb[boolean_arry_bins]

    #Stats BUTA new
    mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
    var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
    stddev_buta_new = np.sqrt(var_buta_new)
    max_buta_new    = np.max(hist_buta_new)    
    sigma_buta      = np.sqrt(hist_buta_new)
    #Stats CARB new
    sigma_carb      = np.sqrt(hist_carb_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_buta)):
        if (sigma_buta[i] == 0):
            sigma_buta[i] = 0.01
    for i in range(len(sigma_carb)):
        if (sigma_carb[i] == 0):
            sigma_carb[i] = 0.01

    #=================================================================================================
    #Params from fitting 
    buta_popt = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]

    #=================================================================================================
    #Final chisq Fitting
    def chisq(sol, x=np.array([]), \
              buta_popt=np.array([]), carb_popt=np.array([]), \
              sigma_buta=np.array([]), sigma_carb=np.array([]), \
              hist_buta=np.array([]), hist_carb=np.array([])):
        #Buta bacground
        gaus_buta_back = buta_popt[2]*np.exp(-1.0*(x - buta_popt[3])**2 / (2*(buta_popt[4]**2)))
        poly_buta      = buta_popt[5]
        buta           = gaus_buta_back + poly_buta

        #Carb background
        gaus_carb_back = carb_popt[2]*np.exp(-1.0*(x - carb_popt[3])**2 / (2*(carb_popt[4]**2)))
        poly_carb      = carb_popt[5]
        carb           = gaus_carb_back + poly_carb

        #each bins
        chisq_list = []
        for i in range(len(x)):
            #nume
            nume = pow(buta[i] - sol*carb[i], 2)
            #deno
            deno = pow(sigma_buta[i] + sigma_carb[i], 2)
            #chisq/bin
            chisq_bin = nume / deno
            #chisq_bin = nume

            #append
            chisq_list.append(chisq_bin)
        #sum chisq/bin
        chisq = np.sum(chisq_list)
        return chisq            

    #chisq initial guess for scale factor
    chi_init_guess = [chi_init_mult[0]*raw_sf]

    #chisq bounds for scale factor
    bounds_chi = [(bounds_chi_mult[0]*raw_sf, bounds_chi_mult[1]*raw_sf)]

    #Minimize
    popt = scipy.optimize.minimize(chisq, x0=chi_init_guess, bounds=bounds_chi, \
                                   args=(bin_centers_new, buta_popt, carb_popt, \
                                         hist_buta_new, hist_carb_new), \
                                   method='L-BFGS-B')

    #=================================================================================================
    #Goodness of fit 
    dof       = (len(bin_centers_new) - 1 ) - len(popt.x)
    chisq_per = np.abs(popt.fun) / dof
    
    #Store redone fits
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]    = popt.x[0]
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.fun"]  = popt.fun
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"] = chisq_per

In [ ]:
#Indi fit plot
def plot_sf_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, \
                 epho_bins=[], cos_bins={}, \
                 buta_fit_dict={}, carb_fit_dict={}, \
                 buta_dict={}, carb_dict={}, \
                 sf_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]            
    
    #SF*carb
    sf                   = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
    chisq_per            = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"]
    df_carb              = carb_dict[epho_keys[epho_bin]][cos_bin]
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
    mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    sf_redo = plt.figure(figsize=(15,10))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=1., range=(range_list[0], range_list[1]), \
             histtype="step", lw=2., \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=1, range=(range_list[0], range_list[1]), \
             histtype="step", lw=2., \
             color="#55A868")
    
    #Plot Scaled Carbon
    plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.5, label="Scaled Carbon", \
            #color="#C44E52")
            #color="#8C0900")
            color="grey")
    
    #Plot fitting
    plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=2.5, \
             color="#DC143C", alpha=1., ls="-", label="buta fit")
             #color="red", alpha=1., ls="-", label="buta fit")
    plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=2.5, \
             color="black", alpha=1, ls="-", label="buta bound fit")            
             #color="#00FFFF", alpha=1, ls="-", label="buta bound fit")            
             #color="#7FFF00", alpha=1, ls="-", label="buta bound fit")            
    plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=2.5, \
             color="#DC143C", alpha=1, ls="--", label="carb fit")
             #color="red", alpha=1, ls="--", label="carb fit")
    plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=2.5, \
             color="black", alpha=1, ls="--", label="carb bound_fit")            
             #color="#00FFFF", alpha=1, ls="--", label="carb bound_fit")            
             #color="#7FFF00", alpha=1, ls="--", label="carb bound_fit")            
    
    ##Butanol free nucleon sharp peak start/end positions
    #plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    #plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #===================================================================================================
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10^2)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
    
    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/100)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))    

    #Texts
    plt.text(0.03, 0.43, " Scale factor {:>14.3f}\n $\chi^2/dof$ {:>23.3f}" \
            .format(sf, chisq_per), \
            fontsize=22, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)
#     #Texts
#     plt.text(0.03, 0.53, " Scale factor {:>14.3f}" \
#             .format(sf), \
#             fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)



    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=20, loc="upper left")

    plt.show()
    sf_redo.savefig("./run_period_1/sf_final_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
##COS=0

epho_bin, cos_bin = 1, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.4])

epho_bin, cos_bin = 2, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[.9, 1.1])

epho_bin, cos_bin = 7, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 12, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 14, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 17, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 18, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

##COS=1

epho_bin, cos_bin = 1, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 7, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 16, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

##COS=2

epho_bin, cos_bin = 7, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

##COS=3

epho_bin, cos_bin = 7, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.195, 1.2])

epho_bin, cos_bin = 12, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.32, 1.34])

epho_bin, cos_bin = 13, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.32, 1.34])

epho_bin, cos_bin = 14, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.32, 1.34])

epho_bin, cos_bin = 15, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.32, 1.34])

##COS=4

epho_bin, cos_bin = 1, 4
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[.565, .6])

epho_bin, cos_bin = 2, 4
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[.9, .91])


##COS=5

epho_bin, cos_bin = 2, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[.96, .97])

epho_bin, cos_bin = 3, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])

epho_bin, cos_bin = 4, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])

epho_bin, cos_bin = 5, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])

epho_bin, cos_bin = 6, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])

epho_bin, cos_bin = 7, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])

epho_bin, cos_bin = 8, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])

epho_bin, cos_bin = 9, 5
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.2])


##COS=6

epho_bin, cos_bin = 2, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1., 1.1])


##COS=7

epho_bin, cos_bin = 2, 7
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.175, 1.2])

epho_bin, cos_bin = 3, 7
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.175, 1.2])



##COS=8

epho_bin, cos_bin = 3, 7
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.175, 1.2])

###### Butanol Free Peak AGAIN - GLOBAL (MUST)

In [ ]:
#Butanol - SF*Carbon Parameter Exraction
def buta_free_params_final(buta_dict={}, carb_dict={}, sf_dict={}, sf_init_dict={}, \
                           epho_bins=[], cos_bins={}):
    #=====================================================================================
    #Dictionary to be returned
    bf_params_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        #list: For each epho bin, containing a list for each angle bin
        bf_1epho_cos_list = []
        
        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(sf_dict[epho_keys[epho_bin]].keys())        
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Corresponding Scale factor for the interested bin
            sf = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]

            #If final sf is nan since discarded bin, use initial sf just to avoid error
            if(np.isnan(sf)):
                sf = sf_init_dict[epho_keys[epho_bin]][cos_bin]

            #=====================================================================================
            #Determine Bin sizes
            bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

            #Make histograms first
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
            hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)
                                
            #=====================================================================================
            #Free butanol peak histograms
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb

            #If any bin in hist_bf is negative, set it to 0
            for i in range(len(hist_bf)):
                if(hist_bf[i] < 0):
                    hist_bf[i] = 0
            
            #Mids for plotting scale carbon HISTOGRAM
            mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

            #Stats for free butanol histogram
            peak_bf  = bins_buta[np.argmax(hist_buta)]
            mean_bf  = np.average(mids, weights=hist_bf)
            var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
            stddv_bf = np.sqrt(var_bf) 
            max_bf   = np.max(hist_bf)

            #=====================================================================================
            #Shorten the fit range
            #    -peak_bf +/- certain amount of range to enclose the peak and 1 background.
            #    -If the fit range is too wide, the background consists of many gaussians, hindering bg fits.
            xmin_fit = peak_bf - 2*stddv_bf
            xmax_fit = peak_bf + 2*stddv_bf
                        
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
            mids_fit          = mids[boolean_arry_bins]
            
            #subset of hist_bf for fitting
            hist_bf_fit = hist_bf[boolean_arry_bins]
            
            #Stats for free butanol histogram
            peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
            mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
            var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
            stddv_bf_fit = np.sqrt(var_bf_fit) 
            max_bf_fit   = np.max(hist_bf_fit)     
            pedestal     = max_bf_fit/20
            
            #=====================================================================================
            #Fit Function (inner)
            def fit_func_combined(x, A, sigma, B, sigma2):
                #Background function
                #Set mu fixed to mean_bf to reduce dof
                gaussian  = A*np.exp(-1.0*(x - mean_bf)**2  / (2*sigma**2))
                #Peak function
                #Set mu fixed to peak_bf to reduce dof
                gaussian2 = B*np.exp(-1.0*(x - peak_bf)**2 / (2*sigma2**2))
                #Final Func
                final_func = gaussian + gaussian2 + pedestal
                return final_func    

            #=====================================================================================
            #Fit
            try:
                params, pconv = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, \
                                          p0=[round(max_bf_fit/2, 3), round(stddv_bf_fit, 3), \
                                              round(max_bf_fit, 3), round(stddv_bf_fit/5, 3)])

                #=====================================================================================
                #Butanol free peak start/end positions
                buta_start_end = [peak_bf - 3*params[3], peak_bf + 3*params[3]]
            
                #Few bins have bad fit -> start and end pos are swapped
                if(buta_start_end[0] > buta_start_end[1]):
                    temp = buta_start_end[0]
                    buta_start_end[0] = buta_start_end[1]
                    buta_start_end[1] = temp
            
            except RuntimeError as err1:
                #If fit is not good, set estimated width for free-nucleon peak. Then change later
                params         = [np.nan, np.nan, np.nan]
                buta_start_end = [peak_bf - 0.02, peak_bf + 0.02]
                
            #=====================================================================================
            #Append mids, bins_bf, hist_bf, and final params for each cos bins to bf_1epho_cos_lisabs
            bf_1epho_cos_list.append([mids, bins_bf, hist_bf, params, mean_bf, peak_bf, buta_start_end, \
                                      mids_fit, pedestal, pconv])
    
        #=========================================================================================
        bf_params_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = bf_1epho_cos_list
        
    return bf_params_dict, epho_bins, cos_bins    

In [ ]:
#bf_params_dist has Bins, x_coordinates, hist_bf, and fitting parameters for each epho/cos_bins 
bf_params_dict, epho_bins, cos_bins = buta_free_params_final(buta_dict, carb_dict, sf_dict, init_sf_2_dict,\
                                                             epho_bins, cos_bins)

###### Butanol Free Peak AGAIN - INDIVIDUAL (MUST)

In [ ]:
#bf_params_final_indi
def buta_free_params_indi(epho_bin=0, cos_bin=0, bf_start=-0.01, bf_end=0.07, 
                         epho_bins=[], cos_bins={}):
    #Keys for epho bins
    epho_keys = sorted(cos_bins.keys())
    
    #Store redone bf ranges
    bf_params_dict[epho_keys[epho_bin]][cos_bin][6] = (bf_start, bf_end)

In [ ]:
#Plot bf indi
def plot_bf_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, \
                 epho_bins=[], cos_bins={}, \
                 buta_fit_dict={}, carb_fit_dict={}, \
                 buta_dict={}, carb_dict={}, \
                 sf_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]            
    
    #SF*carb
    sf                   = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
    chisq_per            = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"]
    df_carb              = carb_dict[epho_keys[epho_bin]][cos_bin]
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
    mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    bf_redo = plt.figure(figsize=(15,10))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=0.9, range=(range_list[0], range_list[1]), \
             color="#55A868")
    
#     #Plot Scaled Carbon
#     plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.3, color="#C44E52", label="Scaled Carbon")
    
#     #Plot fitting
#     plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
#              color="red", alpha=0.7, ls="-", label="buta fit")
#     plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
#              color="blue", alpha=0.7, ls="-", label="buta bound fit")            
#     plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
#              color="red", alpha=0.7, ls="--", label="carb fit")
#     plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
#              color="blue", alpha=0.7, ls="--", label="carb bound_fit")            
    
    #Butanol free nucleon sharp peak start/end positions
    plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=1, lw=2, label="Buta Free-Nucl Peak Start/End pos")
    plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=1, lw=2)

    #===================================================================================================
    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/100)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10^2)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Texts
    plt.text(0.03, 0.53, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
             .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
             fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=20, loc="upper left")

    plt.show()
    bf_redo.savefig("./run_period_1/bf_final_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
##COS=0

epho_bin, cos_bin = 1, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.05, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

##COS=1

epho_bin, cos_bin = 0, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.05, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 1, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.05, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 2, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.05, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

##COS=2

epho_bin, cos_bin = 0, 2
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.0475, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 1, 2
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.0475, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 2, 2
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.0475, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

#COS=3

epho_bin, cos_bin = 0, 3
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.045, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 1, 3
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.045, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 2, 3
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.0475, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 19, 3
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.02, bf_end=0.09, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

#COS=4

epho_bin, cos_bin = 18, 4
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.02, bf_end=0.09, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

#COS=5

epho_bin, cos_bin = 1, 5
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.022, bf_end=0.0475, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 2, 5
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.022, bf_end=0.0475, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

#COS=8

epho_bin, cos_bin = 12, 8
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=0.02, bf_end=0.0675, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

#### DF

In [ ]:
#df inidi
def df_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
              epho_bins=[], cos_bins={}, \
              buta_fit_dict={}, carb_fit_dict={}, \
              sf_dict={}):
    
    #df dict
    df_dict = {}
    
    #keys
    epho_keys     = sorted(cos_bins.keys())
    cos_keys      = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    
        #Dict to store dict or whole cos
        df_1epho_cos_dict = {}    
    
        #keys for each cos_bin to access fit params
        cos_keys_keys = list(buta_fit_dict[epho_keys[epho_bin]].keys())

        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
        
            #=================================================================================================
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

            #Remove Outliers - remove
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Into numpy
            butanol_data = np.array(df_buta)

            #=================================================================================================
            #params from previous steps
            buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
            peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
            buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]

            #Use the bins defined in previous fitting procedure
            #    -mids for whole range of histograms
            bins                    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"] #same bisn for buta&carb       
            mids                    = np.array([0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)])

            #histograms
            hist_buta, buta_bins = np.histogram(butanol_data, bins=bins)

            #=================================================================================================
            #Integrand for buta backgroun
            def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
                gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus1 + gaus2 + poly
                return y    

            #Fit buta bound
            def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus2 + poly
                return y
            
            #Need to have df=np.nan to discard bins
            try:
                #Integrate under fit_buta background
                buta_bg_intg = scipy.integrate.quad(fit_buta_bound, buta_start_end_tpl[0], buta_start_end_tpl[1], \
                                                    args=(buta_popt[0], buta_popt[1], buta_popt[2], \
                                                          buta_popt[3], buta_popt[4], buta_popt[5], peak_buta))

                #=================================================================================================
                #New histrogram for butanol - range of interest
                boolean_arry_bins = (mids >= buta_start_end_tpl[0]) & (mids <= buta_start_end_tpl[1])
                bin_centers_new   = mids[boolean_arry_bins]

                #Subset of buta histogram 
                hist_buta_new = hist_buta[boolean_arry_bins]

                #bin width
                bin_width = bin_centers_new[1] - bin_centers_new[0]

                #Counting BUTA 
                buta_total_sum = np.sum([bin_width * hist_buta_new[i] for i in range(len(hist_buta_new))])

                #=================================================================================================
                #DF calculation
                df = (buta_total_sum - buta_bg_intg[0]) / buta_total_sum

            #For discarded bins
            except ZeroDivisionError as err1:
                df = np.nan
            
            except IndexError as err1:            
                df = np.nan

            #=================================================================================================
            df_1epho_1cos_dict = {"df":df, "buta_bf_intg":buta_bg_intg[0], "buta_total_sum":buta_total_sum, \
                                  "buta_bf_intg_err":buta_bg_intg[1]}

            #Append
            df_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = df_1epho_1cos_dict
            
        #=================================================================================================
        df_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = df_1epho_cos_dict
    
    return df_dict

In [ ]:
#Discard bad bin
def df_discard_bad(epho_bins=[], cos_bins={}, df_dict={}):
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(df_dict[epho_keys[epho_bin]].keys())
        
        #fn to call within next loop for discarding bins
        def discard_bad_bins(epho_bin=0, cos_bin=0):
            df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["df"]             = np.nan
            df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bf_intg"]   = np.nan
            df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_total_sum"] = np.nan

        #=============================================================================================
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
            #epho=0
            if (epho_bin==0 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=1
            elif (epho_bin==1 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=2
            elif (epho_bin==2 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=3
            elif (epho_bin==3 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=4
            elif (epho_bin==4 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=5
            elif (epho_bin==5 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=6
            elif (epho_bin==6 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=7
            elif (epho_bin==7 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=8
            elif (epho_bin==8 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=9
            elif (epho_bin==9 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=10
            elif (epho_bin==10 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=11
            elif (epho_bin==11 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=12
            elif (epho_bin==12 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=13
            elif (epho_bin==13 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==13 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=14
            elif (epho_bin==14 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==14 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=15
            elif (epho_bin==15 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==15 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==15 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=16
            elif (epho_bin==16 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==16 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==16 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==16 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=17
            elif (epho_bin==17 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==17 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==17 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==17 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=18
            elif (epho_bin==18 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==18 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=19
            elif (epho_bin==19 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==19 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)

In [ ]:
#call df_global
df_dict = df_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                    bf_params_dict=bf_params_dict, \
                    init_sf_2_dict=init_sf_2_dict, \
                    epho_bins=epho_bins, cos_bins=cos_bins, \
                    buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                    sf_dict=sf_dict)    

In [ ]:
#Discard bad bins
df_discard_bad(epho_bins, cos_bins, df_dict)

#### Pol States

In [ ]:
#Optimized Version - Vectorization numpy
def comp_pho_Pol_stat(epho, runNum):
    pol_pho_stat = 0
    
    #E_Beam for run ranges
    if   (runNum >= 55521 and runNum <= 55536):
        E_ebeam   = 1.647
    elif (runNum >= 55537 and runNum <= 55552):
        E_ebeam   = 1.647
    elif (runNum >= 55557 and runNum <= 55594):
        E_ebeam   = 1.647
    elif (runNum >= 55604 and runNum <= 55625):
        E_ebeam   = 2.478
    elif (runNum >= 55630 and runNum <= 55678):
        E_ebeam   = 2.478
    elif (runNum >= 56164 and runNum <= 56193):
        E_ebeam   = 2.478
    elif (runNum >= 56196 and runNum <= 56233):
        E_ebeam   = 2.478
        
    #Error for Polarization of E beam for measured run ranges [Steffen AN sec3.1]
    #CORRECT value used since 2021_05
    E_beam_pol     = [.85228, .78523, .79150, .887, .84167, .84725, .86531, .88409, .87753, \
                      .82534, .79450, .80060, .83267, .83248]
    
    E_beam_pol_err = [.142, .1350 , .126, .1480, .1330, .1530, .1380, .1440, .1480, \
                      .1400, .1410, .1400, .1380, .1320]
     
    #Use std of E_beam_pol as ebeam err - a bit skeptical approach. See Steffen's AN
    ebeam_stat_err = np.std(E_beam_pol)
    
    
    #Variables
    x              = epho / E_ebeam
    x2             = x*x
    pol_pho_stat   = np.sqrt((ebeam_stat_err**2) * ((4*x - x2) / (4 - 4*x + 3*x2))**2)
    
    return pol_pho_stat

#Vectorize
vect_comp_pho_Pol_stat = np.vectorize(comp_pho_Pol_stat, otypes=[np.float])
pol_pho_stat           = vect_comp_pho_Pol_stat(df_GMSH["epho"].values, df_GMSH["runNum"].values)

#Append to df
df_GMSH["pol_pho_stat"] = pd.Series(pol_pho_stat, index=df_GMSH.index)

#### MMSQ

In [ ]:
#MMSQ GLOBAL()
def mmsq_global(df=pd.DataFrame(), epho_bins=[], cos_bins={}, bf_params_dict={}):
    
    #dict to contain df partitioned in epho & cos bins + events within bf peak ranges
    mmsq_dict = {}
    
    #keys
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #================================================================================================================
    #Butanol only
    df_buta = df[df["T_fin"] == 0]
    
    #================================================================================================================
    #For each Eg_bon
    for epho_bin, j in zip(range(len(epho_bins)-1), cos_keys):
        #Partition by epho bin
        df_buta_Eg = df_buta[(df_buta["epho"] > epho_bins[epho_bin]) & \
                             (df_buta["epho"] < epho_bins[epho_bin+1])]
    
        #List of dataframes in same epho bins
        buta_epho_list = []

        #========================================================================================================
        #For each cos_bin
        for cos_bin in range(len(cos_bins[j])-1):
            #Partition by cos
            df_buta_Eg_cos = df_buta_Eg \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][cos_bin]) & \
                              (df_buta_Eg["cthe_cm"] < cos_bins[j][cos_bin+1])]

            #Start and end positions of butanol free nucleon peak
            bf_start = bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0]
            bf_end   = bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1]
            
            #========================================================================================================            
            #MMSQ SELECTION
            df_buta_Eg_cos.drop(df_buta_Eg_cos[(df_buta_Eg_cos["el_pc_mmsq_pi0"] < bf_start) | \
                                               (df_buta_Eg_cos["el_pc_mmsq_pi0"] > bf_end)].index, \
                                inplace=True)

            #Append to its epho list
            buta_epho_list.append(df_buta_Eg_cos)

        #========================================================================================================            
        #Append to mmsq_dict for each energy bins
        mmsq_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = buta_epho_list
        
    return mmsq_dict

In [ ]:
#Apply mmsq_global()
mmsq_dict = mmsq_global(df_GMSH, epho_bins, cos_bins, bf_params_dict)

#### HAE

In [ ]:
#HAE func
def HAE(epho_bins=[], cos_bins={}, mmsq_dict={}, df_dict={}):
    
    #HAE dict
    HAE_dict          = {}
    HAE_stat_err_dict = {}
    
    #keys
    epho_keys     = sorted(cos_bins.keys())
    cos_keys      = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #Dict to store dict or whole cos
        HAE_1epho_cos_dict          = {}    
        HAE_stat_err_1epho_cos_dict = {}    
    
        #keys for each cos_bin to access fit params
        cos_keys_keys = list(df_dict[epho_keys[epho_bin]].keys())

        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
        
            #=================================================================================================
            #DF of current bin
            df = mmsq_dict[epho_keys[epho_bin]][cos_bin]
        
            #Target polarization
            pol_tar      = np.abs(df["tar_Pol"].astype(float)).mean()

            #========================================================================================================
            #PARA and ANPA 
            #    -THIS SHOULD VARY DEPENDING ON TARGET POLARIZATION + bm_helicity
            PARA = len(df[(df["tar_Pol_sign"]==-1) & (df["bm_helicity"]==-1)]) + \
                   len(df[(df["tar_Pol_sign"]==1) & (df["bm_helicity"]==1)])
            
            ANPA = len(df[(df["tar_Pol_sign"]==-1) & (df["bm_helicity"]==1)]) + \
                   len(df[(df["tar_Pol_sign"]==1) & (df["bm_helicity"]==-1)])            

            #========================================================================================================
            #Photon Polarization
            pol_pho_avg = df["pol_pho"].astype(float).mean()
            
            #========================================================================================================
            #Dilution facotr
            dil_fact = df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["df"]
            
            #========================================================================================================
            #Compute HAE
            if((PARA + ANPA) == 0):
                HAE = np.nan
            
            #If dil_fact undefineds since discarded bin
            elif(np.isnan(dil_fact)):
                HAE = np.nan
                
            #If good bins - HAE EQUATION HERE!!!
            elif (np.isnan(dil_fact) == False):
                HAE = (1/dil_fact) * (1/pol_pho_avg) * (1/(pol_tar)) * ((ANPA - PARA) / (PARA + ANPA))
                HAE = round(HAE, 4)
                
            #========================================================================================================
            #Statistical Uncertainty 
            #    -Refer to [Brant ex 6.1] for error of mean calc. Better than Bohm
            
            #df stat error
            df_stat_err = df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bf_intg_err"]
            
            #Target polarization error
            pol_tar_err = np.sqrt(np.sum(pow(df["tar_Pol_stat"], 2) / len(df)))
            
            #Photon polarization error
            pol_pho_avg_stat_err = np.sqrt(np.sum(pow(df["pol_pho_stat"], 2) / len(df)))
            
            #Fianl HAE_stat_err compute
            try:
                #W/ df_stat_err
                HAE_stat_err = np.sqrt(pow(HAE, 2)*(pow((df_stat_err/dil_fact), 2) + \
                                                    pow((pol_tar_err/pol_tar), 2) + \
                                                    pow((pol_pho_avg_stat_err/pol_pho_avg), 2) + \
                                                    (4*ANPA*PARA)/((ANPA + PARA)*pow((PARA - ANPA),2))))                
                
            except ZeroDivisionError:
                HAE_stat_err = np.nan
                
            #=================================================================================================
            #Append
            HAE_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])]          = HAE
            HAE_stat_err_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = HAE_stat_err
            
        #=================================================================================================
        HAE_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = HAE_1epho_cos_dict
        HAE_stat_err_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = HAE_stat_err_1epho_cos_dict
    
    return HAE_dict, HAE_stat_err_dict

In [ ]:
#Call HAE()
HAE_dict, HAE_stat_err_dict = HAE(epho_bins=epho_bins, cos_bins=cos_bins, \
                                  mmsq_dict=mmsq_dict, df_dict=df_dict)

#### Save HAE

In [ ]:
# #Write HAE
# HAE_save = open("./results_HAE/binning_1/systematic/HAE_dict.pkl", "wb")
# pickle.dump(HAE_dict, HAE_save)
# HAE_save.close()

In [ ]:
# #Write HAE_stat_err
# HAE_stat_err_save = open("./results_HAE/binning_1/systematic/HAE_stat_err_dict.pkl", "wb")
# pickle.dump(HAE_stat_err_dict, HAE_stat_err_save)
# HAE_stat_err_save.close()

# HAE Compute - HIGH

#### SF

###### Partition - Butanol and Carbon in each bins (MUST)

In [ ]:
#Partition df in epho & cos bins
def part_df(df=pd.DataFrame(), epho_bins=[], cos_bins={}):
    #================================================================================================================
    #Dictionaries to data frames for each energy + angle bins
    buta_dict = {}
    carb_dict = {}
    
    #================================================================================================================
    #For each targets
    df_buta   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 0) & (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    df_carb   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 1) & (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]

    #================================================================================================================
    #For each Eg_bon
    for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
        df_buta_Eg = df_buta[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_buta["epho"] > epho_bins[i]) & (df_buta["epho"] < epho_bins[i+1])]
        
        df_carb_Eg = df_carb[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_carb["epho"] > epho_bins[i]) & (df_carb["epho"] < epho_bins[i+1])]
        
        #List of dataframes in same epho bins
        buta_epho_list = []
        carb_epho_list = []
        
        #========================================================================================================
        #For each cos_bin
        for k in range(len(cos_bins[j])-1):
            df_buta_Eg_cos = df_buta_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][k]) & (df_buta_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            
            df_carb_Eg_cos = df_carb_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_carb_Eg["cthe_cm"] > cos_bins[j][k]) & (df_carb_Eg["cthe_cm"] < cos_bins[j][k+1])]
        
            #REMOVE OUTLIER
            #    -For epho_bin=1, cos_bin==3, error: multiple copies of same event
            if (i==1 and k==3):
                df_carb_Eg_cos.drop(df_carb_Eg_cos[(df_carb_Eg_cos["el_pc_mmsq_pi0"]>-0.2837) & \
                                                   (df_carb_Eg_cos["el_pc_mmsq_pi0"] < -0.28365)].index, \
                                    inplace=True)
        
            #Append to its epho list
            buta_epho_list.append(df_buta_Eg_cos)
            carb_epho_list.append(df_carb_Eg_cos)
            
        #========================================================================================================            
        #Append to buta_dict & carb_dict for each energy bins
        buta_dict["Eg_{:07.3f}".format(epho_bins[i])] = buta_epho_list
        carb_dict["Eg_{:07.3f}".format(epho_bins[i])] = carb_epho_list            
        
    return buta_dict, carb_dict, epho_bins, cos_bins    

#Apply part_df
buta_dict, carb_dict, epho_bins, cos_bins = part_df(df_GMSH, epho_bins=epho_bins, cos_bins=cos_bins)

###### Initial SF function (MUST)

In [ ]:
#Scale Factor phase 1 (MUST)
#    -Returns # of butanol and # of carbon in each cos_bins and epho_bins for mmsq regions of interest 

#=================================================================================================================
#Selecton Range for SF calculation (i.e. [peak - \sig, peak - 2\sig])
#    -#THIS "sf_select_range" WILL BE DETERMINED BY NATURE OF FREE PEAK OF BUTANOL FITTING PARAMETERS
sf_select_range = (1., 1.5)

#SF function
def init_scale_fact_1(df = pd.DataFrame(), epho_bins=[], cos_bins={}):
    
    #================================================================================================================
    #Dictionaries to hold scale factor for each energy + angle bins
    sf_1_dict          = {}
    
    #================================================================================================================
    #For each targets
    df_buta   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 0) & \
                                                                   (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    df_carb   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 1) & \
                                                                   (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    
    #================================================================================================================
    #For each Eg_bon
    for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
        df_buta_Eg = df_buta[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_buta["epho"] > epho_bins[i]) & (df_buta["epho"] < epho_bins[i+1])]
        
        df_carb_Eg = df_carb[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_carb["epho"] > epho_bins[i]) & (df_carb["epho"] < epho_bins[i+1])]
        
        #list: For each epho bin, containing a list for each angle bin
        sf_1epho_cos_list = []
        
        #========================================================================================================
        #For each cos_bin
        for k in range(len(cos_bins[j])-1):
            df_buta_Eg_cos = df_buta_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][k]) & (df_buta_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            df_carb_Eg_cos = df_carb_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_carb_Eg["cthe_cm"] > cos_bins[j][k]) & (df_carb_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            #inner list: For each angle bin, containg a scalar for each mmsq bin
            sf_1epho_1cos_mmsq_list = []
            
            #========================================================================================================
            #Histogram for MMSQ
            buta_data = np.array(df_buta_Eg_cos["el_pc_mmsq_pi0"])
            carb_data = np.array(df_carb_Eg_cos["el_pc_mmsq_pi0"])
            buta_mmsq_hist, buta_bins = np.histogram(buta_data, bins="fd")
            carb_mmsq_hist, carb_bins = np.histogram(carb_data, bins=buta_bins)
            
            #Find Peak of Butanol
            peak = buta_bins[np.argmax(buta_mmsq_hist)]
            
            #Find stddev of Butanol
            mids   = 0.5*(buta_bins[:-1] + buta_bins[1:])
            mean   = np.average(mids, weights=buta_mmsq_hist)
            var    = np.average((mids - mean)**2, weights=buta_mmsq_hist)
            stddev = np.sqrt(var)
            
            #=====================================================================================================
            #Find ranges of mmsq: [\sigma, 2*\sigma] with Butanol
            range_list = [0.035 + sf_select_range[0]*stddev, 0.035 + sf_select_range[1]*stddev]
#             range_list_low = [peak + sf_select_range_low[0]*stddev, peak + sf_select_range_low[1]*stddev]
#             range_list_up  = [peak + sf_select_range_up[0]*stddev, peak + sf_select_range_up[1]*stddev]
            
            #Make New Histogram within defined ranges above: Events within [\sigma, 2*\sigma]
            n_b_i, buta_select_bins = np.histogram(buta_data[(buta_data>=range_list[0]) & (buta_data<=range_list[1])],\
                                                                        bins="fd")
            n_c_i, carb_select_bins = np.histogram(carb_data[(carb_data>=range_list[0]) & (carb_data<=range_list[1])], \
                                                                        bins=buta_select_bins)
#             n_b_i, buta_select_bins = np.histogram(buta_data[((buta_data>=range_list_low[0]) & (buta_data<=range_list_low[1])) | \
#                                                              ((buta_data>=range_list_up[0])  & (buta_data<=range_list_up[1]))], \
#                                                    bins="fd")
#             n_c_i, carb_select_bins = np.histogram(carb_data[((carb_data>=range_list_low[0]) & (carb_data<=range_list_low[1])) | \
#                                                              ((carb_data>=range_list_up[0])  & (carb_data<=range_list_up[1]))], \
#                                                    bins=buta_select_bins)
            
            #========================================================================================================    
            #Append n_b_i and n_c_i lists to sf_1epho_1cos_mmsq_list
            sf_1epho_1cos_mmsq_list.extend([n_b_i.tolist(), n_c_i.tolist()])
            
            #========================================================================================================    
            #Append sf_1epho_1cos_mmsq_list and to sf_1epho_cos_list
            sf_1epho_cos_list.append(sf_1epho_1cos_mmsq_list)

        #========================================================================================================            
        #Append to sf_eg_list to dict
        sf_1_dict["Eg_{:07.3f}".format(epho_bins[i])] = sf_1epho_cos_list
        
    return sf_1_dict, epho_bins, cos_bins

In [ ]:
#Defined Chi-Squared function (MUST)
def chi_sq_fn(x, n_b_i, n_c_i):
    #As defined in Nick's analysis paper
    chi_sq = np.sum(np.power(n_b_i - x*n_c_i, 2) / (n_b_i + (x**2)*n_c_i))
    return chi_sq

In [ ]:
#Scae factor phase 2 (MUST)
#    -Chi-Square miminization to get scale factor
def init_scale_fact_2(sf_1_dict={}, epho_bins=[], cos_bins={}):

    #==========================================================================================================
    #Dictionary to hold final sf values for each epho/cos bins
    sf_2_dict = {}
    
    #Keys of sf_1_dict corresponding to epho bins
    keys = sorted(sf_1_dict.keys())

    #==========================================================================================================
    #For each epho bins
#     for i in range(len(keys)):
    for i, j in zip(range(len(keys)), sorted(cos_bins.keys())):
        
        #list: For each epho bin, containing a list for each angle bin
        sf_1epho_cos_list = []
        
        #======================================================================================================
        #For each cos bins
        for k in range(len(cos_bins[j])-1):
    
            #Minimize using scipy
            result = scipy.optimize.minimize(chi_sq_fn, x0=5, \
                                             args=(np.array(sf_1_dict[keys[i]][k][0]), np.array(sf_1_dict[keys[i]][k][1])), \
                                             method="CG")
            
            if (result.success == True):
                sf = result.x[0]
            elif (result.success == False):
                #Set sf=5 just for now. Come back and change later
                sf = np.nan
            
            #Append final sf for each cos bins to sf_1epho_cos_lisabs
            sf_1epho_cos_list.append(sf)
    
        #==========================================================================================================
        sf_2_dict["Eg_{:07.3f}".format(epho_bins[i])] = sf_1epho_cos_list

    
    return sf_2_dict, epho_bins, cos_bins

In [ ]:
#Apply scale_fact_1 (MUST)
init_sf_1_dict, epho_bins, cos_bins = init_scale_fact_1(df_GMSH, epho_bins, cos_bins)

In [ ]:
#Call scale_fact_2(MUST)
init_sf_2_dict, epho_bins, cos_bins = init_scale_fact_2(init_sf_1_dict, epho_bins, cos_bins)

###### Butanol Free-proton Peak Parameter Extraction (MUST)

In [ ]:
#Butanol - SF*Carbon Parameter Exraction
def buta_free_params(buta_dict={}, carb_dict={}, sf_dict={}, epho_bins=[], cos_bins={}):
    #=====================================================================================
    #Dictionary to be returned
    bf_params_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        #list: For each epho bin, containing a list for each angle bin
        bf_1epho_cos_list = []
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Corresponding Scale factor for the interested bin
            sf = sf_dict[epho_keys[epho_bin]][cos_bin]

            #=====================================================================================
            #Determine Bin sizes
            bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

            #Make histograms first
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
            hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)

            #=====================================================================================
            #If SF is too large that hist_bf have negative value, change sf and compute again
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb
            counter_bad_bin  = 0 
            for i in range(len(hist_bf)):
                if (hist_bf[i] <= 0):
                    counter_bad_bin += 1
            #If # of bins that have larger sf*carb than butanol is more than 1/20 of butanol bins, half the sf
            if (counter_bad_bin >= int(len(bins_bf)/10)):
                #CHANGE the DENOMENATOR if too manyless bins of hist_bf have negative values!!!!
                sf = sf/2.5
                                
            #=====================================================================================
            #Free butanol peak histograms
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb

            #If any bin in hist_bf is negative, set it to 0
            for i in range(len(hist_bf)):
                if(hist_bf[i] < 0):
                    hist_bf[i] = 0
            
            #Mids for plotting scale carbon HISTOGRAM
            mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

            #Stats for free butanol histogram
            #    -Manualy set peak_bf for high_energy
            peak_bf  = 0.035 
            #peak_bf  = bins_buta[np.argmax(hist_buta)]
            mean_bf  = np.average(mids, weights=hist_bf)
            var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
            stddv_bf = np.sqrt(var_bf) 
            max_bf   = np.max(hist_bf)

            #=====================================================================================
            #Shorten the fit range
            #    -peak_bf +/- certain amount of range to enclose the peak and 1 background.
            #    -If the fit range is too wide, the background consists of many gaussians, hindering bg fits.
            xmin_fit = peak_bf - 2*stddv_bf
            xmax_fit = peak_bf + 2*stddv_bf
                        
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
            mids_fit          = mids[boolean_arry_bins]
            
            #subset of hist_bf for fitting
            hist_bf_fit = hist_bf[boolean_arry_bins]
            
            #Stats for free butanol histogram
            peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
            mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
            var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
            stddv_bf_fit = np.sqrt(var_bf_fit) 
            max_bf_fit   = np.max(hist_bf_fit)     
            pedestal     = max_bf_fit/20
            
            #=====================================================================================
            #Fit Function (inner)
            def fit_func_combined(x, A, sigma, B, sigma2):
                #Background function
                #Set mu fixed to mean_bf to reduce dof
                gaussian  = A*np.exp(-1.0*(x - mean_bf)**2  / (2*sigma**2))
                #Peak function
                #Set mu fixed to peak_bf to reduce dof
                gaussian2 = B*np.exp(-1.0*(x - peak_bf)**2 / (2*sigma2**2))
                #Final Func
                final_func = gaussian + gaussian2 + pedestal
                return final_func    

            #=====================================================================================
            #Initial guess
            init_guess = [round(max_bf_fit/2, 3), round(stddv_bf_fit, 3), \
                          round(max_bf_fit, 3), round(stddv_bf_fit/5, 3)]
            
            #Fit
            try:
                params, pconv = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, \
                                          p0=init_guess)

            except RuntimeError:
                #Set empty list
                params  = [np.nan for i in range(len(init_guess))]
                pconv   = [np.nan for i in range(len(init_guess))]                
                
            
            #=====================================================================================
            #Butanol free peak start/end positions
            buta_start_end = (peak_bf - 3*params[3], peak_bf + 3*params[3])

            #=====================================================================================
            #Append mids, bins_bf, hist_bf, and final params for each cos bins to bf_1epho_cos_lisabs
            bf_1epho_cos_list.append([mids, bins_bf, hist_bf, params, mean_bf, peak_bf, buta_start_end, \
                                      mids_fit, pedestal])
    
        #=========================================================================================
        bf_params_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = bf_1epho_cos_list
        
    return bf_params_dict, epho_bins, cos_bins    

In [ ]:
#bf_params_dist has Bins, x_coordinates, hist_bf, and fitting parameters for each epho/cos_bins 
bf_params_dict, epho_bins, cos_bins = buta_free_params(buta_dict, carb_dict, init_sf_2_dict, \
                                                       epho_bins, cos_bins)

###### Butanol Free-proton Peak Parameter Extraction - Individually (MUST)

In [ ]:
#bf_params_final_indi
def buta_free_params_indi(epho_bin=0, cos_bin=0, bf_start=-0.01, bf_end=0.07, 
                         epho_bins=[], cos_bins={}):
    #Keys for epho bins
    epho_keys = sorted(cos_bins.keys())
    
    #Store redone bf ranges
    bf_params_dict[epho_keys[epho_bin]][cos_bin][6] = (bf_start, bf_end)

In [ ]:
#Plot refits
def plot_fixed(epho_bin=0, cos_bin=0, bf_params_dict={}, epho_bins=[], cos_bins={}):

    #Plot size
    fit_redo = plt.figure(figsize=(10,6))
    
    #Keys for epho
    epho_keys = sorted(bf_params_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #Fit function
    def fit_func_combined(x, A, mu, sigma, B, mu2, sigma2, pedestal):
        #Background function
        gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
        #Peak function
        gaussian2 = B*np.exp(-1.0*(x - mu2)**2 / (2*sigma2**2))
        #Pedestal
        pedestal  = pedestal
        #Final Func
        final_func = gaussian + gaussian2 + pedestal 
        return final_func

    #1 fit fucnt
    def fit_func_1(x, A, mu, sigma, pedestal):
        #Background function
        gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
        #Final Func
        final_func = gaussian + pedestal 
        return final_func
    
    #Extracted fit parameters
    A_fit              = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][0]
    sigma_fit          = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][1]
    B_fit              = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][2]
    sigma2_fit         = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][3]
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    pedestal           = bf_params_dict[epho_keys[epho_bin]][cos_bin][8]
    mu_fit             = bf_params_dict[epho_keys[epho_bin]][cos_bin][4]
    mu2_fit            = bf_params_dict[epho_keys[epho_bin]][cos_bin][5]

    #Bins, x_coordinates, hist_bf
    mids        = np.array(bf_params_dict[epho_keys[epho_bin]][cos_bin][0]) 
    mids_fit    = np.array(bf_params_dict[epho_keys[epho_bin]][cos_bin][7]) 
    bins_bf     = bf_params_dict[epho_keys[epho_bin]][cos_bin][1]
    hist_bf     = bf_params_dict[epho_keys[epho_bin]][cos_bin][2]

    #Examine each fit funcations separately
    y_val       = fit_func_combined(mids_fit, \
                                    A_fit, mu_fit, sigma_fit, \
                                    B_fit, mu2_fit, sigma2_fit, \
                                    pedestal)
    y_val_1 = fit_func_1(mids_fit, A_fit, mu_fit, sigma_fit, pedestal)
    y_val_2 = fit_func_1(mids_fit, B_fit, mu2_fit, sigma2_fit, pedestal)

    #Plot each fit function separately
    plt.plot(mids_fit, y_val_1, linewidth=2, label="fit_func_1")
    plt.plot(mids_fit, y_val_2, linewidth=2, label="fit_func_2")

    #Plot butanol free histogram
    plt.bar(mids, hist_bf, width=bins_bf[1]-bins_bf[0], alpha=0.7, label="buta-sf*carb", color="#4C72B0")

    #Plot fitting 
    plt.plot(mids_fit, y_val, linewidth=2, label="Buta free-nucl", color="#C44E52")

    #Butanol free nucleon sharp peak start/end positions
    plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Texts
    plt.text(0.65, 0.65, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
             .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
             fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Labels, ticks, lim
    plt.xlabel("Missing mass sq. $[GeV^2/c^4]$", fontsize=15)
    plt.ylabel("Counts", fontsize=15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    # plt.xlim(-0.3, 0.3)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=15)
        
    #Legends
    plt.legend(fontsize=10, loc="upper right")

    plt.show()
    fit_redo.savefig("./run_period_1/fit_redo_high_{:d}_{:d}".format(epho_bin, cos_bin))    

In [ ]:
##COS=0

epho_bin, cos_bin = 7, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.064, bf_end=0.13, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

###### BUTA fit + CARB fit: Separately - GLOBAL (MUST) 

In [ ]:
#Fit Buta:
def fit_buta_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                    epho_bins=[], cos_bins={}, \
                    left_fit_range=2.5, right_fit_range=2.5, \
                    bin_size_mltp = 0.5, \
#                     buta_init_guess_mltplr = [1, 0.5, 1, 1, 1, 1, 1], \
#                     bounds_buta_mltplr = ([0.5, 0.1, 0, -np.inf, -np.inf, -np.inf, -np.inf], \
#                                           [1.5, 1, np.inf, np.inf, np.inf, np.inf, np.inf])):
                    buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                    bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1])):
    
    #Dict to store final
    buta_fit_dict = {}
    
    #Dict to store err msgs
    err_dict = {}
    cnt      = 0    
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        buta_1epho_cos_dict = {}
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            #    -For high energy set peak_buta=0.035
            peak_buta   = 0.035
            #peak_buta   = buta_bins[np.argmax(hist_buta)]
            mean_buta   = np.average(bin_centers, weights=hist_buta)
            var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
            stddev_buta = np.sqrt(var_buta)
            num_buta    = len(butanol_data)
            max_buta    = np.max(hist_buta)
            
            #=================================================================================================
            #Fitting range
            #left_fit_range  = 2.5
            #right_fit_range = 2.5
            start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                         (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

            #Prevent fits to be delcared out of carbon dist. range
            carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            if (start_end[0] < carb_lims[0]):
                start_end[0] = carb_lims[0]
            if (start_end[1] > carb_lims[1]):
                start_end[1] = carb_lims[1]
            
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            #    -this histogram will be used for final chisq minimization fitting!!!
            hist_buta_new = hist_buta[boolean_arry_bins]

            #Stats BUTA new
            mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
            var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
            stddev_buta_new = np.sqrt(var_buta_new)
            max_buta_new    = np.max(hist_buta_new)    
            sigma_buta      = np.sqrt(hist_buta_new)
        
            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_buta)):
                if (sigma_buta[i] == 0):
                    sigma_buta[i] = 0.01
        
            #=================================================================================================
            #Fit buta
            def fit_buta(x, A, sigma1, B, mu, sigma2, a):
                gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus1 + gaus2 + poly
                return y
            
            #Bounds: needed variables
            bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                                      bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])         
            
            #initial guess + bounds
            buta_init_guess = [buta_init_guess_mltplr[0]*max_buta_new, \
                               buta_init_guess_mltplr[1]*stddev_buta_new, \
                               buta_init_guess_mltplr[2]*max_buta_new, \
                               buta_init_guess_mltplr[3]*peak_buta, \
                               buta_init_guess_mltplr[4]*stddev_buta_new, \
                               buta_init_guess_mltplr[5]*max_buta_new]        
            #Bounds of carb
            bounds_buta = ([bounds_buta_mltplr[0][0]*max_buta_new, bounds_buta_mltplr[0][1]*stddev_buta_new, \
                            bounds_buta_mltplr[0][2]*max_buta_new, bounds_buta_mltplr[0][3]*bf_strt_end_dist + peak_buta, 
                            bounds_buta_mltplr[0][4]*stddev_buta_new, \
                            bounds_buta_mltplr[0][5]*max_buta_new]
                           , \
                           [bounds_buta_mltplr[1][0]*max_buta_new, bounds_buta_mltplr[1][1]*stddev_buta_new, \
                            bounds_buta_mltplr[1][2]*max_buta_new, bounds_buta_mltplr[1][3]*bf_strt_end_dist + peak_buta, 
                            bounds_buta_mltplr[1][4]*stddev_buta_new, \
                            bounds_buta_mltplr[1][5]*max_buta_new])                          

            #Curve_fit
            try:
                buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
                                                                 ydata=hist_buta_new, \
                                                                 p0=buta_init_guess, bounds=bounds_buta, \
                                                                 sigma=sigma_buta, absolute_sigma=True)
            except RuntimeError as err1:
                cnt += 1
                err_dict["{:d}".format(cnt)] = [epho_bin, cos_bin, epho_bins[epho_bin], \
                                                cos_bins[cos_keys[epho_bin]][cos_bin], err1]
                #Set empty list
                buta_popt  = [0 for i in range(len(buta_init_guess))]
                buta_pconv = [0 for i in range(len(buta_init_guess))]
                    
            #=================================================================================================
            #Into dicts
            buta_1epho_1cos_dict = {"bin_centers_new":bin_centers_new, "buta_bins":buta_bins, \
                                   "num_buta":num_buta, "peak_buta":peak_buta, \
                                   "buta_popt":buta_popt, "start_end":start_end, \
                                   "bin_size_mltp":bin_size_mltp, "buta_pconv":buta_pconv} 

            #Append
            buta_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = buta_1epho_1cos_dict

        #=================================================================================================
        #Save to dictionary
        buta_fit_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = buta_1epho_cos_dict
        
    return buta_fit_dict, err_dict, epho_bins, cos_bins

In [ ]:
#Call fit_buta
buta_fit_dict, buta_err_dict, epho_bins, cos_bins = \
fit_buta_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                bf_params_dict=bf_params_dict, \
                init_sf_2_dict=init_sf_2_dict, \
                epho_bins=epho_bins, cos_bins=cos_bins, \
                left_fit_range=2.5, right_fit_range=2.5, \
#                 buta_init_guess_mltplr = [1, 0.5, 1, 1, 1, 1, 1], \
#                 bounds_buta_mltplr = ([0.5, 0.1, 0, 0, 0, 0, 0], \
#                                       [1.5, 1, 0.2, np.inf, np.inf, np.inf, np.inf]))
                buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1]))

In [ ]:
#Fit Buta:
def fit_carb_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                    epho_bins=[], cos_bins={}, \
                    left_fit_range=2.5, right_fit_range=2.5, \
                    bin_size_mltp=0.5, \
                    carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 1], \
                    bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 10, 1, 2, 10, np.inf])):
    
    #Dict to store final
    carb_fit_dict = {}
    
    #Dict to store err msgs
    err_dict = {}
    cnt      = 0
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        carb_1epho_cos_dict = {}
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            #    -For high energy, set peak_buta manually
            peak_buta   = 0.035
            #peak_buta   = buta_bins[np.argmax(hist_buta)]
            mean_buta   = np.average(bin_centers, weights=hist_buta)
            var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
            stddev_buta = np.sqrt(var_buta)
            num_buta    = len(butanol_data)
            max_buta    = np.max(hist_buta)
            #Stats CARB
            peak_carb   = buta_bins[np.argmax(hist_carb)]
            mean_carb   = np.average(bin_centers, weights=hist_carb)
            var_carb    = np.average((bin_centers-mean_carb)**2, weights=hist_carb)
            stddev_carb = np.sqrt(var_carb)
            num_carb    = len(carbon_data)
            max_carb    = np.max(hist_carb)
            
            #=================================================================================================
            #Fitting range
            #left_fit_range  = 2.5
            #right_fit_range = 2.5
            start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                         (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

            #Prevent fits to be delcared out of carbon dist. range
            carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            if (start_end[0] < carb_lims[0]):
                start_end[0] = carb_lims[0]
            if (start_end[1] > carb_lims[1]):
                start_end[1] = carb_lims[1]
            
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            hist_carb_new = hist_carb[boolean_arry_bins]

            #Stats CARB new
            mean_carb_new   = np.average(bin_centers_new, weights=hist_carb_new)
            var_carb_new    = np.average((bin_centers_new-mean_carb_new)**2, weights=hist_carb_new)
            stddev_carb_new = np.sqrt(var_carb_new)
            max_carb_new    = np.max(hist_carb_new)
            sigma_carb      = np.sqrt(hist_carb_new)

            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_carb)):
                if (sigma_carb[i] == 0):
                    sigma_carb[i] = 0.01            
            
            #=================================================================================================
            #Carbon fit
            def fit_carb(x, A, sigma1, B, mu, sigma2, a):
                #remaining ice
                gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
                gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly       = a
                y          = gaussian + gaussian_2 + poly
                return y

            #Bounds: needed variables
            bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                                      bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])

            
            #initial guess + bounds
            carb_init_guess = [carb_init_guess_mltplr[0]*max_carb_new, \
                               carb_init_guess_mltplr[1]*stddev_carb_new, \
                               carb_init_guess_mltplr[2]*max_carb_new, \
                               carb_init_guess_mltplr[3]*peak_carb, \
                               carb_init_guess_mltplr[4]*stddev_carb_new, \
                               carb_init_guess_mltplr[5]*max_carb_new]      
            #Bounds of carb
            bounds_carb = ([bounds_carb_mltplr[0][0]*max_carb_new, bounds_carb_mltplr[0][1]*stddev_carb_new, \
                            bounds_carb_mltplr[0][2]*max_carb_new, bounds_carb_mltplr[0][3]*bf_strt_end_dist + peak_carb, 
                            bounds_carb_mltplr[0][4]*stddev_carb_new, \
                            bounds_carb_mltplr[0][5]*max_carb_new]
                           , \
                           [bounds_carb_mltplr[1][0]*max_carb_new, bounds_carb_mltplr[1][1]*stddev_carb_new, \
                            bounds_carb_mltplr[1][2]*max_carb_new, bounds_carb_mltplr[1][3]*bf_strt_end_dist + peak_carb, 
                            bounds_carb_mltplr[1][4]*stddev_carb_new, \
                            bounds_carb_mltplr[1][5]*max_carb_new])              
            
            try:
                #Curve_Fit carbon
                carb_popt, carb_pconv = scipy.optimize.curve_fit(fit_carb, xdata=bin_centers_new, ydata=hist_carb_new, \
                                                                 p0=carb_init_guess, bounds=bounds_carb, \
                                                                 sigma=sigma_carb, absolute_sigma=True)
            
            except RuntimeError as err1:
                cnt += 1
                err_dict["{:d}".format(cnt)] = [epho_bin, cos_bin, epho_bins[epho_bin], \
                                                cos_bins[cos_keys[epho_bin]][cos_bin], err1]
                #Set empty list
                carb_popt  = [0 for i in range(len(carb_init_guess))]
                carb_pconv = [0 for i in range(len(carb_init_guess))]
                
            #=================================================================================================
            #Into dicts
            carb_1epho_1cos_dict = {"bin_centers_new":bin_centers_new, "carb_bins":carb_bins, \
                                   "num_carb":num_carb, "peak_carb":peak_carb, \
                                   "carb_popt":carb_popt, "start_end":start_end, \
                                   "bin_size_mltp":bin_size_mltp, "carb_pconv":carb_pconv} 

            #Append
            carb_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = carb_1epho_1cos_dict

        #=================================================================================================
        #Save to dictionary
        carb_fit_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = carb_1epho_cos_dict
        
    return carb_fit_dict, err_dict, epho_bins, cos_bins

In [ ]:
#Call fit_carb
carb_fit_dict, carb_err_dict, epho_bins, cos_bins = \
fit_carb_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                bf_params_dict=bf_params_dict, \
                init_sf_2_dict=init_sf_2_dict, \
                epho_bins=epho_bins, cos_bins=cos_bins, \
                left_fit_range=2.5, right_fit_range=2.5, \
                carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
                bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, 1]))

###### BUTA fit + CARB fit: Separately - INDIVIDUAL (MUST) 

In [ ]:
#Buta fit individual
def fit_buta_indi(epho_bin=0, cos_bin=0, epho_bins=[], cos_bins={}, \
                  buta_dict={}, carb_dict={}, \
                  bf_params_dict={}, init_sf_2_dict={}, \
                  left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1, \
                  buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                  bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1]), \
                  new_peak=0):
    
    #keys for epho_bins
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]
    
    #=================================================================================================
    #Into numpy
    butanol_data  = np.array(df_buta)
    carbon_data   = np.array(df_carb)
    #bin_size_mltp = 1

    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    #    -For high energy set peak_buta=0.035
    peak_buta   = 0.035
    #peak_buta   = buta_bins[np.argmax(hist_buta)]
    mean_buta   = np.average(bin_centers, weights=hist_buta)
    var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    num_buta    = len(butanol_data)
    max_buta    = np.max(hist_buta)    
    
    #New peak
    if (new_peak != 0):
        peak_buta = new_peak
    
    #=================================================================================================
    #Fitting range
    #left_fit_range  = 2.5
    #right_fit_range = 2.5
    start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                 (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

    #Prevent fits to be delcared out of carbon dist. range
    carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    if (start_end[0] < carb_lims[0]):
        start_end[0] = carb_lims[0]
    if (start_end[1] > carb_lims[1]):
        start_end[1] = carb_lims[1]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    #    -this histogram will be used for final chisq minimization fitting!!!
    hist_buta_new = hist_buta[boolean_arry_bins]

    #Stats BUTA new
    mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
    var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
    stddev_buta_new = np.sqrt(var_buta_new)
    max_buta_new    = np.max(hist_buta_new)        
    sigma_buta      = np.sqrt(hist_buta_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_buta)):
        if (sigma_buta[i] == 0):
            sigma_buta[i] = 0.01
            
    #=================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Bounds: needed variables
    bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                              bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])         

    #initial guess + bounds
    buta_init_guess = [buta_init_guess_mltplr[0]*max_buta_new, \
                       buta_init_guess_mltplr[1]*stddev_buta_new, \
                       buta_init_guess_mltplr[2]*max_buta_new, \
                       buta_init_guess_mltplr[3]*peak_buta, \
                       buta_init_guess_mltplr[4]*stddev_buta_new, \
                       buta_init_guess_mltplr[5]*max_buta_new]        
    #Bounds of carb
    bounds_buta = ([bounds_buta_mltplr[0][0]*max_buta_new, bounds_buta_mltplr[0][1]*stddev_buta_new, \
                    bounds_buta_mltplr[0][2]*max_buta_new, bounds_buta_mltplr[0][3]*bf_strt_end_dist + peak_buta, 
                    bounds_buta_mltplr[0][4]*stddev_buta_new, \
                    bounds_buta_mltplr[0][5]*max_buta_new]
                   , \
                   [bounds_buta_mltplr[1][0]*max_buta_new, bounds_buta_mltplr[1][1]*stddev_buta_new, \
                    bounds_buta_mltplr[1][2]*max_buta_new, bounds_buta_mltplr[1][3]*bf_strt_end_dist + peak_buta, 
                    bounds_buta_mltplr[1][4]*stddev_buta_new, \
                    bounds_buta_mltplr[1][5]*max_buta_new])

    #Curve_Fit
#     buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
#                                                      ydata=hist_buta_new, \
#                                                      p0=buta_init_guess, bounds=bounds_buta)
    
    buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
                                                     ydata=hist_buta_new, \
                                                     p0=buta_init_guess, bounds=bounds_buta, \
                                                     sigma=sigma_buta, absolute_sigma=True)

    
    #Store redone fits into buta_fit_dict
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"] = bin_centers_new
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]       = buta_bins
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["num_buta"]        = num_buta
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]       = peak_buta
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]       = buta_popt
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_pconv"]      = buta_pconv
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]       = start_end
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]   = bin_size_mltp

In [ ]:
#Buta fit individual
def fit_carb_indi(epho_bin=0, cos_bin=0, epho_bins=[], cos_bins={}, \
                  buta_dict={}, carb_dict={}, \
                  bf_params_dict={}, init_sf_2_dict={}, \
                  left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1, \
                  carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 1], \
                  bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, np.inf]), \
                  new_peak=0):
    
    #keys for epho_bins
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())
    
    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #=================================================================================================
    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]
    
    #=================================================================================================
    #Into numpy
    butanol_data  = np.array(df_buta)
    carbon_data   = np.array(df_carb)
    #bin_size_mltp = 1
    
    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    #    -For high energy, set peak_buta manually
    peak_buta   = 0.035
    #peak_buta   = buta_bins[np.argmax(hist_buta)]
    mean_buta   = np.average(bin_centers, weights=hist_buta)
    var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    num_buta    = len(butanol_data)
    max_buta    = np.max(hist_buta)    
    #Stats CARB
    peak_carb   = buta_bins[np.argmax(hist_carb)]
    mean_carb   = np.average(bin_centers, weights=hist_carb)
    var_carb    = np.average((bin_centers-mean_carb)**2, weights=hist_carb)
    stddev_carb = np.sqrt(var_carb)
    num_carb    = len(carbon_data)
    max_carb    = np.max(hist_carb) 
    
    #New peak
    if (new_peak != 0):
        peak_buta = new_peak
        peak_carb = new_peak
    
    #=================================================================================================
    #Fitting range
    #left_fit_range  = 2.5
    #right_fit_range = 2.5
    start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                 (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

    #Prevent fits to be delcared out of carbon dist. range
    carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    if (start_end[0] < carb_lims[0]):
        start_end[0] = carb_lims[0]
    if (start_end[1] > carb_lims[1]):
        start_end[1] = carb_lims[1]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    hist_carb_new = hist_carb[boolean_arry_bins]

    #Stats CARB new
    mean_carb_new   = np.average(bin_centers_new, weights=hist_carb_new)
    var_carb_new    = np.average((bin_centers_new-mean_carb_new)**2, weights=hist_carb_new)
    stddev_carb_new = np.sqrt(var_carb_new)
    max_carb_new    = np.max(hist_carb_new)       
    sigma_carb      = np.sqrt(hist_carb_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_carb)):
        if (sigma_carb[i] == 0):
            sigma_carb[i] = 0.01
    
    #=================================================================================================
    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Bounds: needed variables
    bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                              bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])


    #initial guess + bounds
    carb_init_guess = [carb_init_guess_mltplr[0]*max_carb_new, \
                       carb_init_guess_mltplr[1]*stddev_carb_new, \
                       carb_init_guess_mltplr[2]*max_carb_new, \
                       carb_init_guess_mltplr[3]*peak_carb, \
                       carb_init_guess_mltplr[4]*stddev_carb_new, \
                       carb_init_guess_mltplr[5]*max_carb_new]        
    #Bounds of carb
    bounds_carb = ([bounds_carb_mltplr[0][0]*max_carb_new, bounds_carb_mltplr[0][1]*stddev_carb_new, \
                    bounds_carb_mltplr[0][2]*max_carb_new, bounds_carb_mltplr[0][3]*bf_strt_end_dist + peak_carb, 
                    bounds_carb_mltplr[0][4]*stddev_carb_new, \
                    bounds_carb_mltplr[0][5]*max_carb_new]
                   , \
                   [bounds_carb_mltplr[1][0]*max_carb_new, bounds_carb_mltplr[1][1]*stddev_carb_new, \
                    bounds_carb_mltplr[1][2]*max_carb_new, bounds_carb_mltplr[1][3]*bf_strt_end_dist + peak_carb, 
                    bounds_carb_mltplr[1][4]*stddev_carb_new, \
                    bounds_carb_mltplr[1][5]*max_carb_new])  

    #Curve_Fit
    carb_popt, carb_pconv = scipy.optimize.curve_fit(fit_carb, xdata=bin_centers_new, \
                                                     ydata=hist_carb_new, \
                                                     p0=carb_init_guess, bounds=bounds_carb, \
                                                     sigma=sigma_carb, absolute_sigma=True)

    #Store redone fits into carb_fit_dict
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"] = bin_centers_new
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_bins"]       = carb_bins
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["num_carb"]        = num_carb
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]       = peak_carb
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]       = carb_popt
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_pconv"]      = carb_pconv
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]       = start_end
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]   = bin_size_mltp    

In [ ]:
#Indi fit plot
def plot_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, epho_bins=[], cos_bins={}, \
              buta_fit_dict={}, carb_fit_dict={}, buta_dict={}, carb_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    fit_redo = plt.figure(figsize=(10,6))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]        
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=0.9, range=(range_list[0], range_list[1]), \
             color="#55A868")
    
    #Plot fitting
    plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
             color="red", alpha=0.7, ls="-", label="buta fit")
    plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
             color="blue", alpha=0.7, ls="-", label="buta bound fit")            
    plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
             color="red", alpha=0.7, ls="--", label="carb fit")
    plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
             color="blue", alpha=0.7, ls="--", label="carb bound_fit")            
    
    ##Butanol free nucleon sharp peak start/end positions
    #plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    #plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #===================================================================================================
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=15)
    plt.ylabel("Counts", fontsize=15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    ##Texts
    #plt.text(0.03, 0.53, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
    #         .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
    #         fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=15)
        
    #Legends
    plt.legend(fontsize=10, loc="upper left")

    plt.show()
    fit_redo.savefig("./run_period_1/sf_fit_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
# ##COS=0

epho_bin, cos_bin = 0, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0.5, 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 1, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 2, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 3, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.5, 1, 0.25, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 4, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 5, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 6, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 7, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.4, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)


# ##COS=1

epho_bin, cos_bin = 0, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0.5, 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 2, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 3, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 4, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.6, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 5, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 6, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 7, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 8, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 9, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

# # ##COS=2

epho_bin, cos_bin = 2, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 5, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

# # ##COS=5

epho_bin, cos_bin = 4, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 5, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

# # ##COS=6

epho_bin, cos_bin = 1, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 2, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)


######  CHISQ min for final SF - GLOBAL (MUST)

In [ ]:
#sf global
def sf_final(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
             epho_bins=[], cos_bins={}, \
             buta_fit_dict={}, carb_fit_dict={}, \
             chi_init_mult=[1], bounds_chi_mult=[0, 2]):

    #Dict to store final sf
    sf_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        sf_1epho_cos_dict = {}
    
        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())
    
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):

            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
        
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_size_mltp = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]    
            bin_num       = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            peak_buta   = buta_bins[np.argmax(hist_buta)]
                       
            #=================================================================================================
            #Fitting range            
            start_end = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]

            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            #    -this histogram will be used for final chisq minimization fitting!!!
            hist_buta_new = hist_buta[boolean_arry_bins]
            hist_carb_new = hist_carb[boolean_arry_bins]

            #Stats BUTA new
            mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
            var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
            stddev_buta_new = np.sqrt(var_buta_new)
            max_buta_new    = np.max(hist_buta_new)    
            sigma_buta      = np.sqrt(hist_buta_new)
            #Stats CARB new
            sigma_carb      = np.sqrt(hist_carb_new)
        
            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_buta)):
                if (sigma_buta[i] == 0):
                    sigma_buta[i] = 0.01
            for i in range(len(sigma_carb)):
                if (sigma_carb[i] == 0):
                    sigma_carb[i] = 0.01

            #=================================================================================================
            #Params from fitting 
            buta_popt = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
            carb_popt = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
                    
            #=================================================================================================
            #Final chisq Fitting
            def chisq(sol, x=np.array([]), \
                      buta_popt=np.array([]), carb_popt=np.array([]), \
                      #sigma_buta=np.array([]), sigma_carb=np.array([]), \
                      hist_buta=np.array([]), hist_carb=np.array([])):
                #Buta bacground
                gaus_buta_back = buta_popt[2]*np.exp(-1.0*(x - buta_popt[3])**2 / (2*(buta_popt[4]**2)))
                poly_buta      = buta_popt[5]
                buta           = gaus_buta_back + poly_buta
                
                #Carb background
                gaus_carb_back = carb_popt[2]*np.exp(-1.0*(x - carb_popt[3])**2 / (2*(carb_popt[4]**2)))
                poly_carb      = carb_popt[5]
                carb           = gaus_carb_back + poly_carb
                
                #each bins
                chisq_list = []
                for i in range(len(x)):
                    #nume
                    nume = pow(buta[i] - sol*carb[i], 2)
                    #deno
                    #deno = pow(sigma_buta[i] + sigma_carb[i], 2)
                    deno = buta[i] + pow(sol, 2)*carb[i]
                    
                    #chisq/bin
                    chisq_bin = nume / deno
                    #chisq_bin = nume

                    #append
                    chisq_list.append(chisq_bin)
                #sum chisq/bin
                chisq = np.sum(chisq_list)
                return chisq            
            
            #chisq initial guess for scale factor
            chi_init_guess = [chi_init_mult[0]*raw_sf]
            
            #chisq bounds for scale factor
            bounds_chi = [(bounds_chi_mult[0]*raw_sf, bounds_chi_mult[1]*raw_sf)]
            
            #Minimize
            popt = scipy.optimize.minimize(chisq, x0=chi_init_guess, bounds=bounds_chi, \
                                           args=(bin_centers_new, buta_popt, carb_popt, \
                                                 hist_buta_new, hist_carb_new), \
                                           method='L-BFGS-B')

            #=================================================================================================
            #Goodness of fit 
            dof       = (len(bin_centers_new) - 1 ) - len(popt.x)
            chisq_per = np.abs(popt.fun) / dof
            
            #=================================================================================================
            #SF Stat uncertainties (NEED WORK)
            sf_stat_err_fit      = 0
            sf_stat_err_chisq    = 0
            
            sf_stat_err = sf_stat_err_fit + sf_stat_err_chisq

            #=================================================================================================
            sf_1epho_1cos_dict = {"popt.x": popt.x[0], "popt.fun":popt.fun, "chisq_per":chisq_per, \
                                  "popt":popt, "sf_stat_err":sf_stat_err}
            
            #Append
            sf_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = sf_1epho_1cos_dict
        
        #=================================================================================================
        #Save to dictionary
        sf_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = sf_1epho_cos_dict
            
    return sf_dict, epho_bins, cos_bins                

In [ ]:
#Discard bad bin
def sf_discard_bad(epho_bins=[], cos_bins={}, sf_dict={}):
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(sf_dict[epho_keys[epho_bin]].keys())
        
        #fn to call within next loop for discarding bins
        def discard_bad_bins(epho_bin=0, cos_bin=0):
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"] = np.nan
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.fun"]  = np.nan
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"] = np.nan

        #=============================================================================================
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
            #epho=0
            if (epho_bin==0 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=1
            elif (epho_bin==1 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=2
            elif (epho_bin==2 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=3
            elif (epho_bin==3 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=4
            elif (epho_bin==4 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

                
            #epho=5
            elif (epho_bin==5 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=6
            elif (epho_bin==6 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
            
            #epho=7
            elif (epho_bin==7 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=8
            elif (epho_bin==8 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==3):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=9
            elif (epho_bin==9 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==2):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==3):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)                

In [ ]:
#call sf_dict global
sf_dict, epho_bins, cos_bins = sf_final(buta_dict=buta_dict, carb_dict=carb_dict, \
                                        bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
                                        epho_bins=epho_bins, cos_bins=cos_bins, \
                                        buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                                        chi_init_mult=[1], bounds_chi_mult=[0.1, 2.])

#Discard bad bins
sf_discard_bad(epho_bins, cos_bins, sf_dict)

######  CHISQ min for final SF - INDIVIDUAL (MUST)

In [ ]:
#sf_final individual
def sf_final_indi(epho_bin=0, cos_bin=0, \
                  buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                  epho_bins=[], cos_bins={}, \
                  buta_fit_dict={}, carb_fit_dict={}, \
                  chi_init_mult=[1], bounds_chi_mult=[0, 2]):
    
    #keys for epho_bins
    epho_keys      = sorted(carb_dict.keys())
    cos_keys       = sorted(cos_bins.keys())
    cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

    #=================================================================================================
    #Into numpy
    butanol_data = np.array(df_buta)
    carbon_data  = np.array(df_carb)

    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_size_mltp = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]    
    bin_num       = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    peak_buta   = buta_bins[np.argmax(hist_buta)]

    #=================================================================================================
    #Fitting range            
    start_end = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    #    -this histogram will be used for final chisq minimization fitting!!!
    hist_buta_new = hist_buta[boolean_arry_bins]
    hist_carb_new = hist_carb[boolean_arry_bins]

    #Stats BUTA new
    mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
    var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
    stddev_buta_new = np.sqrt(var_buta_new)
    max_buta_new    = np.max(hist_buta_new)    
    sigma_buta      = np.sqrt(hist_buta_new)
    #Stats CARB new
    sigma_carb      = np.sqrt(hist_carb_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_buta)):
        if (sigma_buta[i] == 0):
            sigma_buta[i] = 0.01
    for i in range(len(sigma_carb)):
        if (sigma_carb[i] == 0):
            sigma_carb[i] = 0.01

    #=================================================================================================
    #Params from fitting 
    buta_popt = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]

    #=================================================================================================
    #Final chisq Fitting
    def chisq(sol, x=np.array([]), \
              buta_popt=np.array([]), carb_popt=np.array([]), \
              sigma_buta=np.array([]), sigma_carb=np.array([]), \
              hist_buta=np.array([]), hist_carb=np.array([])):
        #Buta bacground
        gaus_buta_back = buta_popt[2]*np.exp(-1.0*(x - buta_popt[3])**2 / (2*(buta_popt[4]**2)))
        poly_buta      = buta_popt[5]
        buta           = gaus_buta_back + poly_buta

        #Carb background
        gaus_carb_back = carb_popt[2]*np.exp(-1.0*(x - carb_popt[3])**2 / (2*(carb_popt[4]**2)))
        poly_carb      = carb_popt[5]
        carb           = gaus_carb_back + poly_carb

        #each bins
        chisq_list = []
        for i in range(len(x)):
            #nume
            nume = pow(buta[i] - sol*carb[i], 2)
            #deno
            deno = pow(sigma_buta[i] + sigma_carb[i], 2)
            #chisq/bin
            chisq_bin = nume / deno
            #chisq_bin = nume

            #append
            chisq_list.append(chisq_bin)
        #sum chisq/bin
        chisq = np.sum(chisq_list)
        return chisq            

    #chisq initial guess for scale factor
    chi_init_guess = [chi_init_mult[0]*raw_sf]

    #chisq bounds for scale factor
    bounds_chi = [(bounds_chi_mult[0]*raw_sf, bounds_chi_mult[1]*raw_sf)]

    #Minimize
    popt = scipy.optimize.minimize(chisq, x0=chi_init_guess, bounds=bounds_chi, \
                                   args=(bin_centers_new, buta_popt, carb_popt, \
                                         hist_buta_new, hist_carb_new), \
                                   method='L-BFGS-B')

    #=================================================================================================
    #Goodness of fit 
    dof       = (len(bin_centers_new) - 1 ) - len(popt.x)
    chisq_per = np.abs(popt.fun) / dof
    
    #Store redone fits
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]    = popt.x[0]
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.fun"]  = popt.fun
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"] = chisq_per

In [ ]:
#Indi fit plot
def plot_sf_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, \
                 epho_bins=[], cos_bins={}, \
                 buta_fit_dict={}, carb_fit_dict={}, \
                 buta_dict={}, carb_dict={}, \
                 sf_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]            
    
    #SF*carb
    sf                   = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
    chisq_per            = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"]
    df_carb              = carb_dict[epho_keys[epho_bin]][cos_bin]
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
    mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    sf_redo = plt.figure(figsize=(15,10))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=1., range=(range_list[0], range_list[1]), \
             histtype="step", lw=2., \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=1, range=(range_list[0], range_list[1]), \
             histtype="step", lw=2., \
             color="#55A868")
    
    #Plot Scaled Carbon
    plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.5, label="Scaled Carbon", \
            #color="#C44E52")
            #color="#8C0900")
            color="grey")
    
    #Plot fitting
    plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=2.5, \
             color="#DC143C", alpha=1., ls="-", label="buta fit")
             #color="red", alpha=1., ls="-", label="buta fit")
    plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=2.5, \
             color="black", alpha=1, ls="-", label="buta bound fit")            
             #color="#00FFFF", alpha=1, ls="-", label="buta bound fit")            
             #color="#7FFF00", alpha=1, ls="-", label="buta bound fit")            
    plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=2.5, \
             color="#DC143C", alpha=1, ls="--", label="carb fit")
             #color="red", alpha=1, ls="--", label="carb fit")
    plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=2.5, \
             color="black", alpha=1, ls="--", label="carb bound_fit")            
             #color="#00FFFF", alpha=1, ls="--", label="carb bound_fit")            
             #color="#7FFF00", alpha=1, ls="--", label="carb bound_fit")            
    
    ##Butanol free nucleon sharp peak start/end positions
    #plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    #plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #===================================================================================================
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
    
    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/10)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))    

    #Texts
    plt.text(0.03, 0.43, " Scale factor {:>14.3f}\n $\chi^2/dof$ {:>23.3f}" \
            .format(sf, chisq_per), \
            fontsize=22, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)
#     #Texts
#     plt.text(0.03, 0.53, " Scale factor {:>14.3f}" \
#             .format(sf), \
#             fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)



    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=20, loc="upper left")

    plt.show()
    sf_redo.savefig("./run_period_1/sf_final_redo_high_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
##COS=0

epho_bin, cos_bin = 0, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.4, 1.45])

epho_bin, cos_bin = 1, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 2, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.35, 1.4])

epho_bin, cos_bin = 3, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 4, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

epho_bin, cos_bin = 5, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 7, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 8, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

##COS=1

epho_bin, cos_bin = 0, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 1, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 2, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 3, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 4, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 5, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 6, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])


epho_bin, cos_bin = 7, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 8, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.25, 1.3])

epho_bin, cos_bin = 9, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

##COS=2

epho_bin, cos_bin = 0, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 1, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 2, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 3, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 4, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 5, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 6, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])


epho_bin, cos_bin = 7, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 8, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1., 1.1])

##COS=3

epho_bin, cos_bin = 1, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 4, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 5, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

##COS=4

epho_bin, cos_bin = 6, 4
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])


##COS=6

epho_bin, cos_bin = 0, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

epho_bin, cos_bin = 1, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

epho_bin, cos_bin = 2, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 3, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])


##COS=7

epho_bin, cos_bin = 2, 7
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

##COS=8

epho_bin, cos_bin = 0, 8
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

###### Butanol Free Peak AGAIN - GLOBAL (MUST)

In [ ]:
#Butanol - SF*Carbon Parameter Exraction
def buta_free_params_final(buta_dict={}, carb_dict={}, sf_dict={}, sf_init_dict={}, \
                           epho_bins=[], cos_bins={}):
    #=====================================================================================
    #Dictionary to be returned
    bf_params_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        #list: For each epho bin, containing a list for each angle bin
        bf_1epho_cos_list = []
        
        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(sf_dict[epho_keys[epho_bin]].keys())        
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Corresponding Scale factor for the interested bin
            sf = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]

            #If final sf is nan since discarded bin, use initial sf just to avoid error
            if(np.isnan(sf)):
                sf = sf_init_dict[epho_keys[epho_bin]][cos_bin]

            #=====================================================================================
            #Determine Bin sizes
            bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

            #Make histograms first
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
            hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)
                                
            #=====================================================================================
            #Free butanol peak histograms
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb

            #If any bin in hist_bf is negative, set it to 0
            for i in range(len(hist_bf)):
                if(hist_bf[i] < 0):
                    hist_bf[i] = 0
            
            #Mids for plotting scale carbon HISTOGRAM
            mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

            #Stats for free butanol histogram
            peak_bf  = bins_buta[np.argmax(hist_buta)]
            mean_bf  = np.average(mids, weights=hist_bf)
            var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
            stddv_bf = np.sqrt(var_bf) 
            max_bf   = np.max(hist_bf)

            #=====================================================================================
            #Shorten the fit range
            #    -peak_bf +/- certain amount of range to enclose the peak and 1 background.
            #    -If the fit range is too wide, the background consists of many gaussians, hindering bg fits.
            xmin_fit = peak_bf - 2*stddv_bf
            xmax_fit = peak_bf + 2*stddv_bf
                        
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
            mids_fit          = mids[boolean_arry_bins]
            
            #subset of hist_bf for fitting
            hist_bf_fit = hist_bf[boolean_arry_bins]
            
            #Stats for free butanol histogram
            peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
            mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
            var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
            stddv_bf_fit = np.sqrt(var_bf_fit) 
            max_bf_fit   = np.max(hist_bf_fit)     
            pedestal     = max_bf_fit/20
            
            #=====================================================================================
            #Fit Function (inner)
            def fit_func_combined(x, A, sigma, B, sigma2):
                #Background function
                #Set mu fixed to mean_bf to reduce dof
                gaussian  = A*np.exp(-1.0*(x - mean_bf)**2  / (2*sigma**2))
                #Peak function
                #Set mu fixed to peak_bf to reduce dof
                gaussian2 = B*np.exp(-1.0*(x - peak_bf)**2 / (2*sigma2**2))
                #Final Func
                final_func = gaussian + gaussian2 + pedestal
                return final_func    

            #=====================================================================================
            #Fit
            try:
                params, pconv = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, \
                                          p0=[round(max_bf_fit/2, 3), round(stddv_bf_fit, 3), \
                                              round(max_bf_fit, 3), round(stddv_bf_fit/5, 3)])

                #=====================================================================================
                #Butanol free peak start/end positions
                buta_start_end = [peak_bf - 3*params[3], peak_bf + 3*params[3]]
            
                #Few bins have bad fit -> start and end pos are swapped
                if(buta_start_end[0] > buta_start_end[1]):
                    temp = buta_start_end[0]
                    buta_start_end[0] = buta_start_end[1]
                    buta_start_end[1] = temp
            
            except RuntimeError as err1:
                #If fit is not good, set estimated width for free-nucleon peak. Then change later
                params         = [np.nan, np.nan, np.nan]
                buta_start_end = [peak_bf - 0.02, peak_bf + 0.02]
                
            #=====================================================================================
            #Append mids, bins_bf, hist_bf, and final params for each cos bins to bf_1epho_cos_lisabs
            bf_1epho_cos_list.append([mids, bins_bf, hist_bf, params, mean_bf, peak_bf, buta_start_end, \
                                      mids_fit, pedestal, pconv])
    
        #=========================================================================================
        bf_params_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = bf_1epho_cos_list
        
    return bf_params_dict, epho_bins, cos_bins    

In [ ]:
#bf_params_dist has Bins, x_coordinates, hist_bf, and fitting parameters for each epho/cos_bins 
bf_params_dict, epho_bins, cos_bins = buta_free_params_final(buta_dict, carb_dict, sf_dict, init_sf_2_dict,\
                                                             epho_bins, cos_bins)

###### Butanol Free Peak AGAIN - INDIVIDUAL (MUST)

In [ ]:
#bf_params_final_indi
def buta_free_params_indi(epho_bin=0, cos_bin=0, bf_start=-0.01, bf_end=0.07, 
                         epho_bins=[], cos_bins={}):
    #Keys for epho bins
    epho_keys = sorted(cos_bins.keys())
    
    #Store redone bf ranges
    bf_params_dict[epho_keys[epho_bin]][cos_bin][6] = (bf_start, bf_end)

In [ ]:
#Plot bf indi
def plot_bf_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, \
                 epho_bins=[], cos_bins={}, \
                 buta_fit_dict={}, carb_fit_dict={}, \
                 buta_dict={}, carb_dict={}, \
                 sf_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]            
    
    #SF*carb
    sf                   = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
    chisq_per            = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"]
    df_carb              = carb_dict[epho_keys[epho_bin]][cos_bin]
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
    mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    bf_redo = plt.figure(figsize=(15,10))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=0.9, range=(range_list[0], range_list[1]), \
             color="#55A868")
    
#     #Plot Scaled Carbon
#     plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.3, color="#C44E52", label="Scaled Carbon")
    
#     #Plot fitting
#     plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
#              color="red", alpha=0.7, ls="-", label="buta fit")
#     plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
#              color="blue", alpha=0.7, ls="-", label="buta bound fit")            
#     plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
#              color="red", alpha=0.7, ls="--", label="carb fit")
#     plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
#              color="blue", alpha=0.7, ls="--", label="carb bound_fit")            
    
    #Butanol free nucleon sharp peak start/end positions
    plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=1, lw=2, label="Buta Free-Nucl Peak Start/End pos")
    plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=1, lw=2)

    #===================================================================================================
    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/100)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10^2)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Texts
    plt.text(0.6, 0.7, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
             .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
             fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=17.5, loc="upper right")

    plt.show()
    bf_redo.savefig("./run_period_1/bf_final_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
##COS=0

epho_bin, cos_bin = 0, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 2, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 6, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 7, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

##COS=1

epho_bin, cos_bin = 3, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 8, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 9, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

##COS=2

epho_bin, cos_bin = 8, 2
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


##COS=3

epho_bin, cos_bin = 7, 3
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


##COS=4

epho_bin, cos_bin = 6, 4
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


##COS=5

epho_bin, cos_bin = 5, 5
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


##COS=7

epho_bin, cos_bin = 2, 7
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


##COS=7

epho_bin, cos_bin = 0, 8
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


#### DF

#### Pol States

#### MMSQ

#### HAE

#### Save HAE

In [ ]:
# #Write HAE
# HAE_save = open("./results_HAE/binning_1/systematic/high_energy/HAE_dict.pkl", "wb")
# pickle.dump(HAE_dict, HAE_save)
# HAE_save.close()

In [ ]:
# #Write HAE_stat_err
# HAE_stat_err_save = open("./results_HAE/binning_1/systematic/high_energy/HAE_stat_err_dict.pkl", "wb")
# pickle.dump(HAE_stat_err_dict, HAE_stat_err_save)
# HAE_stat_err_save.close()

###### Save HAE

# Syst. Uncerntainty

###### Read HAE

In [ ]:
#==================== LOW =======================================================
#Read HAE
HAE_save  = open("./results_HAE/binning_1/systematic/HAE_dict.pkl", "rb")
HAE_dict  = pickle.load(HAE_save)
HAE_save.close()

#Read HAE_stat_err
HAE_stat_err_save  = open("./results_HAE/binning_1/systematic/HAE_stat_err_dict.pkl", "rb")
HAE_stat_err_dict  = pickle.load(HAE_stat_err_save)
HAE_stat_err_save.close()

#==================== LOW =======================================================
#Read HAE
HAE_save  = open("./results_HAE/binning_1/systematic/high_energy/HAE_dict.pkl", "rb")
HAE_dict  = pickle.load(HAE_save)
HAE_save.close()

#Read HAE_stat_err
HAE_stat_err_save  = open("./results_HAE/binning_1/systematic/high_energy/HAE_stat_err_dict.pkl", "rb")
HAE_stat_err_dict  = pickle.load(HAE_stat_err_save)
HAE_stat_err_save.close()

###### Sum HAE Orig

###### Sum HAE - Varied

###### \Delta E + \sigma(\Delta E)

###### Plot \Delta E + \sigma

###### Save \Delta E + \sigma

# ETC